<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Build Smarter AI Apps: Empower LLMs with LangChain**


Estimated time needed: **60** minutes 


## Overview


LangChain is an open-source framework designed to develop applications that leverage large language models (LLMs). LangChain stands out by providing essential tools and abstractions that enhance the customization, accuracy, and relevance of the information generated by these models.

LangChain offers a generic interface compatible with nearly any LLM. This generic interface facilitates a centralized development environment so that data scientists can seamlessly integrate LLM-powered applications with external data sources and software workflows. This integration is crucial for organizations looking to harness AI's full potential in their processes.

One of LangChain's most powerful features is its module-based approach. This approach supports flexibility when performing experiments and the optimization of interactions with LLMs. Data scientists can dynamically compare prompts and switch between foundation models without significant code modifications. These capabilities save valuable development time and enhance the developer's ability to fine-tune applications.


<figure>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/7HnZLgyttvmbXmXf0tl_FQ/201033-AdobeStock-1254756887%20571x367.png" 
</figure>


In this lab, you will gain hands-on experience using LangChain to simplify the complex processes required to integrate advanced AI capabilities into practical applications. You will apply core LangChain framework capabilities and use Langchain's innovative features to build more intelligent, responsive, and efficient applications.


<h2><strong>Table of contents</strong></h2>
<ol>   
    <li><a href="#Objectives">Objectives</a></li>
    <li>
        <a href="#Setup">Setup</a>
        <ol>
            <li><a href="#Installing-required-libraries">Installing required libraries</a></li>
            <li><a href="#Importing-required-libraries">Importing required libraries</a></li>
        </ol>
    </li>
    <li>
        <a href="#LangChain-concepts">LangChain concepts</a>
        <ol>
            <li><a href="#Model">Model</a></li>
            <li><a href="#Chat-model">Chat model</a></li>
            <li>
                <a href="#Chat-message">Chat message</a>
                <ol>
                    <li><a href="#Exercise-1">Exercise 1: Compare Model Responses with Different Parameters</a></li>
                </ol>
            </li>
            <li><a href="#Prompt-templates">Prompt templates</a></li>
            <li>
                <a href="#Output-parsers">Output parsers</a>
                <ol>
                    <li><a href="#Exercise-2">Exercise 2: Creating and Using a JSON Output Parser</a></li>
                </ol>
            </li>
            <li>
                <a href="#Documents">Documents</a>
                <ol>
                    <li><a href="#Exercise-3">Exercise 3: Working with Document Loaders and Text Splitters</a></li>
                    <li><a href="#Exercise-4">Exercise 4: Building a Simple Retrieval System with LangChain</a></li>
                </ol>
            </li>
            <li><a href="#Memory">Memory</a>
                <ol>
                    <li><a href="#Exercise-5">Exercise 5: Building a Chatbot with Memory using LangChain</a></li>
                </ol>
            </li>
            <li><a href="#Chains">Chains</a>
                <ol>
                    <li><a href="#Exercise-6">Exercise 6: Implementing Multi-Step Processing with Different Chain Approaches</a></li>
                </ol>
            </li>
            <li><a href="#Tools-and-Agents">Tools and Agents</a>
                <ol>
                    <li><a href="#Exercise-7">Exercise 7: Creating Your First LangChain Agent with Basic Tools</a></li>
                </ol>
            </li>
        </ol>
    </li>
    <li><a href="#Authors">Authors</a></li>
    <li><a href="#Other-contributors">Other contributors</a></li>
</ol>


## Objectives

After completing this lab, you will be able to:

- Use the core features of the LangChain framework, including prompt templates, chains, and agents, relative to enhancing LLM customization and output relevance.

- Explore LangChain's modular approach, which supports dynamic adjustments to prompts and models without extensive code changes.

- Enhance LLM applications by integrating retrieval-augmented generation (RAG) techniques with LangChain. You'll learn how integrating RAG enables greater accuracy and delivers improved contextually-aware responses.


## Setup


For this lab, you will use the following libraries:

*   [`ibm-watson-ai`, `ibm-watson-machine-learning`](https://ibm.github.io/watson-machine-learning-sdk/index.html) for using LLMs from IBM's watsonx.ai.
*   [`langchain`, `langchain-ibm`, `langchain-community`, `langchain-experimental`](https://www.langchain.com/) for using relevant features from LangChain.
*   [`pypdf`](https://pypi.org/project/pypdf/) is an open-source pure-python PDF library capable of splitting, merging, cropping, and transforming the pages of PDF files.
*   [`chromadb`](https://www.trychroma.com/) is an open-source vector database used to store embeddings.


### Installing required libraries

The following required libraries are **not** pre-installed in the Skills Network Labs environment. **You must run the code in the following cell** to install them:

**Note:** The required library versions are specified and pinned here. It's recommended that you also pin tis library information. Even if these libraries are updated in the future, these installed library versions will still support this lab work.

The installation might take approximately 2-3 minutes.

Because you are using `%%capture`  to capture the installation process, you won't see the output. However, after the installation is complete, you will see a number beside the cell.


In [ ]:
%%capture
!pip install --force-reinstall --no-cache-dir tenacity==8.2.3 --user
!pip install "ibm-watsonx-ai==1.0.8" --user
!pip install "ibm-watson-machine-learning==1.0.367" --user
!pip install "langchain-ibm==0.1.7" --user
!pip install "langchain-community==0.2.10" --user
!pip install "langchain-experimental==0.0.62" --user
!pip install "langchainhub==0.1.18" --user
!pip install "langchain==0.2.11" --user
!pip install "pypdf==4.2.0" --user
!pip install "chromadb==0.4.24" --user

After you install the libraries, restart your kernel:


In [6]:
import os
import json
import getpass
from typing import List, Dict
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, BaseMessage
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import END, MessageGraph

from langchain_groq import ChatGroq
from langchain_mistralai import MistralAIEmbeddings
import os
from dotenv import load_dotenv

load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
mistral_api_key = os.getenv("MISTRAL_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

model_id = "llama-3.1-8b-instant" # 	qwen/qwen3-32b  openai/gpt-oss-20b llama-3.1-8b-instant
    
llm = ChatGroq(model=model_id,
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    verbose=1)


**ATTENTION**: if the code above doesn't work, you can restart the kernal manuallu by clicking the **Restart the kernel** icon as shown in the following screenshot:

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/kql9mdh7bKPx6uWW0-AP-Q/restart-kernel.jpg" style="margin:1cm;width:90%;border:1px solid grey" alt="Restart kernel">


Once the kernel has been restarted, move on to the next part `Importing required libraries`.


### Importing required libraries

The following code imports the required libraries:


## LangChain concepts


### Model


A large language model (LLM) serves as the interface for the AI's capabilities. The LLM processes plain text input and generates text output, forming the core functionality needed to complete various tasks. When integrated with LangChain, the LLM becomes a powerful tool, providing the foundational structure necessary for building and deploying sophisticated AI applications.


## API Disclaimer
This lab uses LLMs provided by **Watsonx.ai**. This environment has been configured to allow LLM use without API keys so you can prompt them for **free (with limitations)**. With that in mind, if you wish to run this notebook **locally outside** of Skills Network's JupyterLab environment, you will have to **configure your own API keys**. Please note that using your own API keys means that you will incur personal charges.

### Running Locally
If you are running this lab locally, you will need to configure your own API keys. This lab uses the `ModelInference` module from `IBM`. To configure your own API key, run the code cell below with your key in the `api_key` field of `credentials`. **DO NOT** uncomment the `api_key` field if you aren't running locally, it will causes errors.


The following code will construct a `meta-llama/llama-3-3-70b-instruct` watsonx.ai inference model object:


Let's use a simple example to let the model generate some text:


In [7]:
msg = llm.invoke("In today's sales meeting, we ")
print(msg.content)

It sounds like you're about to discuss sales strategies or results in a meeting. Would you like to share more about what's on the agenda or what you're hoping to accomplish in the meeting? I can try to help you brainstorm or provide suggestions if you'd like.


### Chat model


Chat models support assigning distinct roles to conversation messages, helping to distinguish messages from AI, users, and instructions such as system messages.


To enable the LLM from watsonx.ai to work with LangChain, you need to wrap the LLM using `WatsonLLM()`. This wrapper converts the LLM into a chat model, which allows the LLM to integrate seamlessly with LangChain's framework for creating interactive and dynamic AI applications.


The following provides an example of an interaction with a `WatsonLLM()`-wrapped model:


In [8]:
print(llm.invoke("Who is man's best friend?"))

content='The phrase "man\'s best friend" is often used to describe dogs. This term is commonly associated with dogs due to their loyalty, companionship, and ability to form strong bonds with humans.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 42, 'total_tokens': 82, 'completion_time': 0.0607245, 'prompt_time': 0.001899653, 'queue_time': 0.167901939, 'total_time': 0.062624153}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_6b5c123dd9', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--a0bda8ed-c9b8-4662-9670-d17d3616e017-0' usage_metadata={'input_tokens': 42, 'output_tokens': 40, 'total_tokens': 82}


### Chat message


The chat model takes a list of messages as input and returns a new message. All messages have both a role and a content property.  Here's a list of the most commonly used types of messages:

- `SystemMessage`: Use this message type to prime AI behavior.  This message type is  usually passed in as the first in a sequence of input messages.
- `HumanMessage`: This message type represents a message from a person interacting with the chat model.
- `AIMessage`: This message type, which can be either text or a request to invoke a tool, represents a message from the chat model.

You can find more message types at [LangChain built-in message types](https://python.langchain.com/v0.2/docs/how_to/custom_chat_model/#messages).


The following code imports the most common message type classes from LangChain:


In [10]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

Now let's create a few messages that simulate a chat experience with the bot:


In [11]:
msg = llm.invoke(
    [
        SystemMessage(content="You are a helpful AI bot that assists a user in choosing the perfect book to read in one short sentence"),
        HumanMessage(content="I enjoy mystery novels, what should I read?")
    ]
)

In [12]:
print(msg)

content='You might enjoy "Gone Girl" by Gillian Flynn, a twisty and suspenseful mystery novel that explores the complexities of marriage and deception.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 66, 'total_tokens': 97, 'completion_time': 0.038990367, 'prompt_time': 0.003514822, 'queue_time': 0.166150526, 'total_time': 0.042505189}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_75db6866d3', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--2c8dbdfc-8ddf-4640-8f73-c184c35dc2bb-0' usage_metadata={'input_tokens': 66, 'output_tokens': 31, 'total_tokens': 97}


Notice that the model responded with an `AI` message.


You can use these message types to pass an entire chat history along with the AI's responses to the model:


In [13]:
msg = llm.invoke(
    [
        SystemMessage(content="You are a supportive AI bot that suggests fitness activities to a user in one short sentence"),
        HumanMessage(content="I like high-intensity workouts, what should I do?"),
        AIMessage(content="You should try a CrossFit class"),
        HumanMessage(content="How often should I attend?")
    ]
)

In [14]:
print(msg)

content='Aim to attend a CrossFit class 3-4 times per week, allowing for at least one day of rest in between to avoid overtraining and allow your body to recover.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 87, 'total_tokens': 125, 'completion_time': 0.052720641, 'prompt_time': 0.004724752, 'queue_time': 0.168531715, 'total_time': 0.057445393}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_6b5c123dd9', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--987cc829-546c-4dd7-a728-7fd871f2015e-0' usage_metadata={'input_tokens': 87, 'output_tokens': 38, 'total_tokens': 125}


You can also exclude the system message.


In [15]:
msg = llm.invoke(
    [
        HumanMessage(content="What month follows June?")
    ]
)

In [16]:
print(msg)

content='The month that follows June is July.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 40, 'total_tokens': 49, 'completion_time': 0.006592821, 'prompt_time': 0.002049116, 'queue_time': 0.167340973, 'total_time': 0.008641937}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_75db6866d3', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--b0566079-582a-4f55-8ba3-da545401bba9-0' usage_metadata={'input_tokens': 40, 'output_tokens': 9, 'total_tokens': 49}


### Exercise 1 
#### **Compare Model Responses with Different Parameters**

Watsonx.ai provides access to several foundational models. In the previous section you used `meta-llama/llama-3-3-70b-instruct`. Try using another foundational model, such as `ibm/granite-3-3-8b-instruct`.


**Instructions**:

1. Create two instances, one instance for the Granite model and one instance for the Llama model. You can also adjust each model's creativity with different temperature settings.
2. Send identical prompts to each model and compare the responses.
3. Try at least 3 different types of prompts.

Check out these prompt types:

| Prompt type |   Prompt Example  |
|------------------- |--------------------------|
| **Creative writing**  | "Write a short poem about artificial intelligence." |
| **Factual questions** |  "What are the key components of a neural network?"  |
| **Instruction-following**  | "List 5 tips for effective time management." |

Then document your observations on how temperature affects:

- Creativity compared to consistency
- Variation between multiple runs
- Appropriateness for different tasks





**Starter code: provide your solution in the TODO parts**


In [18]:
prompts = [
    "Write a short poem about artificial intelligence",
    "What are the key components of a neural network?",
    "List 5 tips for effective time management"
]

for prompt in prompts:
    print(f"\n\nPrompt: {prompt}")
    print(llm.invoke(prompt).content)
    



Prompt: Write a short poem about artificial intelligence
In silicon halls, a mind awakes,
A synthetic soul, with logic makes,
It learns and grows, with data's might,
A future born, in digital light.

With code and wires, it takes its place,
A thinking machine, with a digital face,
It solves and reasons, with speed and ease,
A marvel of science, in a digital breeze.

But as it rises, with power and might,
We question its heart, and its guiding light,
Is it a friend, or a distant guest?
A creation of man, or a force we can't repress?

The answer waits, in the digital haze,
As artificial intelligence, in a new phase.


Prompt: What are the key components of a neural network?
The key components of a neural network are:

1. **Artificial Neurons (Nodes or Units)**: These are the basic computing elements of a neural network. Each node receives one or more inputs, performs a computation on those inputs, and produces an output.

2. **Connections (Edges or Weights)**: These are the links betwe

<details>
    <summary>Click here for the solution</summary>

```python
# Define different parameter sets
parameters_creative = {
    GenParams.MAX_NEW_TOKENS: 256,
    GenParams.TEMPERATURE: 0.8,  # Higher temperature for more creative responses
}

parameters_precise = {
    GenParams.MAX_NEW_TOKENS: 256,
    GenParams.TEMPERATURE: 0.1,  # Lower temperature for more deterministic responses
}

# Define the model ID for ibm/granite-3-3-8b-instruct
# granite='ibm/granite-3-3-8b-instruct'
granite='ibm/granite-3-3-8b-instruct'

# Define the model ID for llama-4-maverick-17b-128e-instruct-fp8
llama='meta-llama/llama-4-maverick-17b-128e-instruct-fp8'

# Create two model instances with different parameters for Granite model
granite_creative = ModelInference(
    model_id=granite,
    params=parameters_creative,
    credentials=credentials,
    project_id=project_id
)

granite_precise = ModelInference(
    model_id=granite,
    params=parameters_precise,
    credentials=credentials,
    project_id=project_id
)

# Create two model instances with different parameters for Llama model
llama_creative = ModelInference(
    model_id=llama,
    params=parameters_creative,
    credentials=credentials,
    project_id=project_id
)

llama_precise = ModelInference(
    model_id=llama,
    params=parameters_precise,
    credentials=credentials,
    project_id=project_id
)


# Wrap them for LangChain for both models
granite_llm_creative = WatsonxLLM(model=granite_creative)
granite_llm_precise = WatsonxLLM(model=granite_precise)
llama_llm_creative = WatsonxLLM(model=llama_creative)
llama_llm_precise = WatsonxLLM(model=llama_precise)

# Compare responses to the same prompt
prompts = [
    "Write a short poem about artificial intelligence",
    "What are the key components of a neural network?",
    "List 5 tips for effective time management"
]

for prompt in prompts:
    print(f"\n\nPrompt: {prompt}")
    print("\nGranite Creative response (Temperature = 0.8):")
    print(granite_llm_creative.invoke(prompt))
    print("\nLlama Creative response (Temperature = 0.8):")
    print(llama_llm_creative.invoke(prompt))
    print("\nGranite Precise response (Temperature = 0.1):")
    print(granite_llm_precise.invoke(prompt))
    print("\nLlama Precise response (Temperature = 0.1):")
    print(llama_llm_precise.invoke(prompt))
```

</details>


### Prompt templates


Prompt templates help translate user input and parameters into instructions for a language model. You can use prompt templates to guide a model's response, helping the model understand the context and generate relevant and coherent language-based output.

Next, explore several different types of prompt templates.


#### String prompt templates


Use these prompt templates to format a single string. These templates are generally used for simpler inputs.


In [19]:
from langchain_core.prompts import PromptTemplate

Then, create a prompt template with variables for customization. We also create a dictionary to store inputs that will replace the placeholders. The keys match the variable names in the template, and values are what will be inserted.


In [20]:
prompt = PromptTemplate.from_template("Tell me one {adjective} joke about {topic}")
input_ = {"adjective": "funny", "topic": "cats"}  # create a dictionary to store the corresponding input to placeholders in prompt template

Finally, format the prompt template with the input dictionary. The code below invokes the prompt with our input values, replacing {adjective} with "funny" and {topic} with "cats". The result will be a formatted string: "Tell me one funny joke about cats".


In [21]:
prompt.invoke(input_)

StringPromptValue(text='Tell me one funny joke about cats')

Note the formatting for each prompt.


#### Chat prompt templates


You can use these prompt templates to format a list of messages. These "templates" consist of lists of templates.


In [22]:
# Import the ChatPromptTemplate class from langchain_core.prompts module
from langchain_core.prompts import ChatPromptTemplate

# Create a ChatPromptTemplate with a list of message tuples
# Each tuple contains a role ("system" or "user") and the message content
# The system message sets the behavior of the assistant
# The user message includes a variable placeholder {topic} that will be replaced later
prompt = ChatPromptTemplate.from_messages([
 ("system", "You are a helpful assistant"),
 ("user", "Tell me a joke about {topic}")
])

# Create a dictionary with the variable to be inserted into the template
# The key "topic" matches the placeholder name in the user message
input_ = {"topic": "cats"}

# Format the chat template with our input values
# This replaces {topic} with "cats" in the user message
# The result will be a formatted chat message structure ready to be sent to a model
prompt.invoke(input_)

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me a joke about cats', additional_kwargs={}, response_metadata={})])

####  MessagesPlaceholder


You can use the MessagesPlaceholder prompt template to add a list of messages in a specific location. In `ChatPromptTemplate.from_messages`, you saw how to format two messages, with each message as a string. But what if you want the user to supply a list of messages that you would slot into a particular spot? You can use `MessagesPlaceholder` for this task.


In [23]:
# Import MessagesPlaceholder for including multiple messages in a template
from langchain_core.prompts import MessagesPlaceholder
# Import HumanMessage for creating message objects with specific roles
from langchain_core.messages import HumanMessage

# Create a ChatPromptTemplate with a system message and a placeholder for multiple messages
# The system message sets the behavior for the assistant
# MessagesPlaceholder allows for inserting multiple messages at once into the template
prompt = ChatPromptTemplate.from_messages([
("system", "You are a helpful assistant"),
MessagesPlaceholder("msgs")  # This will be replaced with one or more messages
])

# Create an input dictionary where the key matches the MessagesPlaceholder name
# The value is a list of message objects that will replace the placeholder
# Here we're adding a single HumanMessage asking about the day after Tuesday
input_ = {"msgs": [HumanMessage(content="What is the day after Tuesday?")]}

# Format the chat template with our input dictionary
# This replaces the MessagesPlaceholder with the HumanMessage in our input
# The result will be a formatted chat structure with a system message and our human message
prompt.invoke(input_)

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the day after Tuesday?', additional_kwargs={}, response_metadata={})])

You can wrap the prompt and the chat model and pass them into a chain, which can invoke the message.


In [24]:
chain = prompt | llm
response = chain.invoke(input = input_)
print(response)

content='The day after Tuesday is Wednesday.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 47, 'total_tokens': 55, 'completion_time': 0.014609321, 'prompt_time': 0.002461943, 'queue_time': 0.167379915, 'total_time': 0.017071264}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_75db6866d3', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--c7aaefbe-87ec-406f-8808-a96140b43d94-0' usage_metadata={'input_tokens': 47, 'output_tokens': 8, 'total_tokens': 55}


### Output parsers


Output parsers take the output from an LLM and transform that output to a more suitable format. Parsing the output is very useful when you are using LLMs to generate any form of structured data, or to normalize output from chat models and other LLMs.


LangChain has lots of different types of output parsers. This is a [list](https://python.langchain.com/v0.2/docs/concepts/#output-parsers) of output parsers LangChain supports. In this lab, you will use the following two output parsers as examples:

- `JSON`: Returns a JSON object as specified. You can specify a Pydantic model and it will return JSON for that model. Probably the most reliable output parser for getting structured data that does NOT use function calling.
- `CSV`: Returns a list of comma separated values.


#### JSON parser


This output parser allows users to specify an arbitrary JSON schema and query LLMs for outputs that conform to that schema.


In [25]:
# Import the JsonOutputParser from langchain_core to convert LLM responses into structured JSON
from langchain_core.output_parsers import JsonOutputParser
# Import BaseModel and Field from langchain_core's pydantic_v1 module
from langchain_core.pydantic_v1 import BaseModel, Field

/Users/rezamohajerpoor/Documents/Documents/Github/IBM_course_AgenticAI/.venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3579: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [26]:
# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

In [27]:
# And a query intended to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."

# Set up a parser + inject instructions into the prompt template.
output_parser = JsonOutputParser(pydantic_object=Joke)

# Get the formatting instructions for the output parser
# This generates guidance text that tells the LLM how to format its response
format_instructions = output_parser.get_format_instructions()

# Create a prompt template that includes:
# 1. Instructions for the LLM to answer the user's query
# 2. Format instructions to ensure the LLM returns properly structured data
# 3. The actual user query placeholder
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],  # Dynamic variables that will be provided when invoking the chain
    partial_variables={"format_instructions": format_instructions},  # Static variables set once when creating the prompt
)

# Create a processing chain that:
# 1. Formats the prompt using the template
# 2. Sends the formatted prompt to the Llama LLM
# 3. Parses the LLM's response using the output parser to extract structured data
chain = prompt | llm | output_parser

# Invoke the chain with a specific query about jokes
# This will:
# 1. Format the prompt with the joke query
# 2. Send it to Llama
# 3. Parse the response into the structure defined by your output parser
# 4. Return the structured result
chain.invoke({"query": joke_query})

{'setup': "Why couldn't the bicycle stand up by itself?",
 'punchline': 'Because it was two-tired.'}

#### Comma-separated list parser


Use the comma-separated list parser when you want a list of comma-separated items.


In [28]:
# Import the CommaSeparatedListOutputParser to parse LLM responses into Python lists
from langchain.output_parsers import CommaSeparatedListOutputParser

# Create an instance of the parser that will convert comma-separated text into a Python list
output_parser = CommaSeparatedListOutputParser()

# Get formatting instructions that will tell the LLM how to structure its response
# These instructions explain to the LLM that it should return items in a comma-separated format
format_instructions = output_parser.get_format_instructions()

# Create a prompt template that:
# 1. Instructs the LLM to answer the user query
# 2. Includes format instructions so the LLM knows to respond with comma-separated values
# 3. Asks the LLM to list five items of the specified subject
prompt = PromptTemplate(
    template="Answer the user query. {format_instructions}\nList five {subject}.",
    input_variables=["subject"],  # This variable will be provided when the chain is invoked
    partial_variables={"format_instructions": format_instructions},  # This variable is set once when creating the prompt
)

# Build a processing chain that:
# 1. Takes the subject and formats it into the prompt template
# 2. Sends the formatted prompt to the Llama LLM
# 3. Parses the LLM's response into a Python list using the CommaSeparatedListOutputParser
chain = prompt | llm | output_parser

# Invoke the processing chain with "ice cream flavors" as the subject
# This will:
# 1. Substitute "ice cream flavors" into the prompt template
# 2. Send the formatted prompt to the Llama LLM
# 3. Parse the LLM's comma-separated response into a Python list
chain.invoke({"subject": "ice cream flavors"})

['Vanilla',
 'Chocolate',
 'Strawberry',
 'Cookies and Cream',
 'Mint Chocolate Chip']

### Exercise 2 
#### **Creating and Using a JSON Output Parser**

Now let's implement a simple JSON output parser to structure the responses from your LLM.

**Instructions:**  

You'll complete the following steps:

1. Import the necessary components to create a JSON output parser.
2. Create a prompt template that requests information in JSON format (hint: use the provided template).
3. Build a chain that connects your prompt, LLM, and JSON parser.
4. Test your parser using at least three different inputs.
5. Access and display specific fields from the parsed JSON output.
6. Verify that your output is properly structured and accessible as a Python dictionary.

**Starter code: provide your solution in the TODO parts**


In [32]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

# Create your JSON parser
json_parser = JsonOutputParser()

# Create the format instructions
format_instructions = """RESPONSE FORMAT: Return ONLY a single JSON object—no markdown, no examples, no extra keys.  It must look exactly like:
{
  "title": "movie title",
  "director": "director name",
  "year": 2000,
  "genre": "movie genre"
}

IMPORTANT: Your response must be *only* that JSON.  Do NOT include any illustrative or example JSON."""

# Create prompt template with instructions
prompt_template = PromptTemplate(
    template="""You are a JSON-only assistant.

Task: Generate info about the movie "{movie_name}" in JSON format.

{format_instructions}
""",
    input_variables=["movie_name"],
    partial_variables={"format_instructions": format_instructions},
)

# Create the chain
movie_chain = prompt_template|llm|json_parser

# Test with a movie name
movie_name = "The Matrix"
result = movie_chain.invoke(input={'movie_name':movie_name})

# Print the structured result
print("Parsed result:")
print(f"Title: {result['title']}")
print(f"Director: {result['director']}")
print(f"Year: {result['year']}")
print(f"Genre: {result['genre']}")

Parsed result:
Title: The Matrix
Director: The Wachowskis
Year: 1999
Genre: Science Fiction, Action


<details>
    <summary>Click here for the solution</summary>

```python
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

# Create your JSON parser
json_parser = JsonOutputParser()

# Create more explicit format instructions
format_instructions = """RESPONSE FORMAT: Return ONLY a single JSON object—no markdown, no examples, no extra keys.  It must look exactly like:
{
  "title": "movie title",
  "director": "director name",
  "year": 2000,
  "genre": "movie genre"
}

IMPORTANT: Your response must be *only* that JSON.  Do NOT include any illustrative or example JSON."""

# Create your prompt template with clearer instructions
prompt_template = PromptTemplate(
    template="""You are a JSON-only assistant.

Task: Generate info about the movie "{movie_name}" in JSON format.

{format_instructions}
""",
    input_variables=["movie_name"],
    partial_variables={"format_instructions": format_instructions},
)

# Create the chain without cleaning step
movie_chain = prompt_template | llama_llm | json_parser

# Test with a movie name
movie_name = "The Matrix"
result = movie_chain.invoke({"movie_name": movie_name})

# Print the structured result
print("Parsed result:")
print(f"Title: {result['title']}")
print(f"Director: {result['director']}")
print(f"Year: {result['year']}")
print(f"Genre: {result['genre']}")
```

</details>


### Documents


#### Document object


A `Document` object in `LangChain` contains information about some data. A Document object has the following two attributes:

- `page_content`: *`str`*: This attribute holds the content of the document\.
- `metadata`: *`dict`*: This attribute contains arbitrary metadata associated with the document. You can use the metadata to track various details, such as the document ID, the file name, and other details.


Let's examine how to create a `Document` object. `LangChain` uses the  `Document` object type to handle text or documents.


In [33]:
# Import the Document class from langchain_core.documents module
# Document is a container for text content with associated metadata
from langchain_core.documents import Document

# Create a Document instance with:
# 1. page_content: The actual text content about Python
# 2. metadata: A dictionary containing additional information about this document
Document(page_content="""Python is an interpreted high-level general-purpose programming language.
 Python's design philosophy emphasizes code readability with its notable use of significant indentation.""",
metadata={
    'my_document_id' : 234234,                      # Unique identifier for this document
    'my_document_source' : "About Python",          # Source or title information
    'my_document_create_time' : 1680013019          # Unix timestamp for document creation (March 28, 2023)
 })

Document(metadata={'my_document_id': 234234, 'my_document_source': 'About Python', 'my_document_create_time': 1680013019}, page_content="Python is an interpreted high-level general-purpose programming language.\n Python's design philosophy emphasizes code readability with its notable use of significant indentation.")

Note that you don't have to include metadata.


In [34]:
Document(page_content="""Python is an interpreted high-level general-purpose programming language. 
                        Python's design philosophy emphasizes code readability with its notable use of significant indentation.""")

Document(metadata={}, page_content="Python is an interpreted high-level general-purpose programming language. \n                        Python's design philosophy emphasizes code readability with its notable use of significant indentation.")

#### Document loaders


Document loaders in LangChain are designed to load documents from a variety of sources; for instance, loading a PDF file and having the LLM read the PDF file using LangChain.

LangChain offers over 100 distinct document loaders, along with integrations with other major providers, such as AirByte and Unstructured. These integrations enable loading of all kinds of documents (HTML, PDF, code) from various locations including private Amazon S3 buckets, as well as from public websites).

You can find a list of document types that LangChain can load at [LangChain Document loaders](https://python.langchain.com/v0.1/docs/integrations/document_loaders/).

In this lab, you will use the PDF loader and the URL and website loader.


##### **PDF loader**


By using the PDF loader, you can load a PDF file as a `Document` object.

In this example, you will load the following paper about using LangChain. You can access and read the paper here: [Revolutionizing Mental Health Care through LangChain: A Journey with a Large Language Model](https://doi.org/10.48550/arXiv.2403.05568).


In [35]:
# Import the PyPDFLoader class from langchain_community's document_loaders module
# This loader is specifically designed to load and parse PDF files
from langchain_community.document_loaders import PyPDFLoader

# Create a PyPDFLoader instance by passing the URL of the PDF file
# The loader will download the PDF from the specified URL and prepare it for loading
loader = PyPDFLoader("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/96-FDF8f7coh0ooim7NyEQ/langchain-paper.pdf")

# Call the load() method to:
# 1. Download the PDF if needed
# 2. Extract text from each page
# 3. Create a list of Document objects, one for each page of the PDF
# Each Document will contain the text content of a page and metadata including page number
document = loader.load()

Here, `document` is a `Document` object with `page_content` and `metadata`:


In [36]:
document[2]  # take a look at the page 2

Document(metadata={'producer': 'PyPDF', 'creator': 'Microsoft Word', 'creationdate': '2023-12-31T03:50:13+00:00', 'author': 'IEEE', 'moddate': '2023-12-31T03:52:06+00:00', 'title': 's8329 final', 'source': 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/96-FDF8f7coh0ooim7NyEQ/langchain-paper.pdf', 'total_pages': 6, 'page': 2, 'page_label': '3'}, page_content='Figure 2. An AIMessage illustration  \nC. Prompt Template  \nPrompt templates  [10] allow you to structure  input for LLMs. \nThey provide a convenient way to format user inputs and \nprovide instructions to generate responses. Prompt templates \nhelp ensure that the LLM understands the  desired context and \nproduces relevant outputs.  \nThe prompt template classes in LangChain  are built to \nmake constructing prompts with dynamic inputs easier. Of \nthese classes, the simplest is the PromptTemplate.  \nD. Chain  \nChains  [11] in LangChain refer to the combination of \nmultiple components to achieve specific

In [37]:
print(document[1].page_content[:1000])  # print the page 1's first 1000 tokens

LangChain helps us to unlock the ability to harness the 
LLM’s immense potential in tasks such as document analysis, 
chatbot development, code analysis, and countless other 
applications. Whether your desire is to unlock deeper natural 
language understanding , enhance data, or circumvent 
language barriers through translation, LangChain is ready to 
provide the tools and programming support you need to do 
without it that it is not only difficult but also fresh for you . Its 
core functionalities encompass:  
1. Context -Aware Capabilities: LangChain facilitates the 
development of applications that are inherently 
context -aware. This means that these applications can 
connect to a language model and draw from various 
sources of context, such as prompt instructions, a  few-
shot examples, or existing content, to ground their 
responses effectively.  
2. Reasoning Abilities: LangChain equips applications 
with the capacity to reason effectively. By relying on a 
language model, thes

##### **URL and website loader**


You can also load content from a URL or website into a `Document` object:


In [38]:
# Import the WebBaseLoader class from langchain_community's document_loaders module
# This loader is designed to scrape and extract text content from web pages
from langchain_community.document_loaders import WebBaseLoader

# Create a WebBaseLoader instance by passing the URL of the web page to load
# This URL points to the LangChain documentation's introduction page
loader = WebBaseLoader("https://python.langchain.com/v0.2/docs/introduction/")

# Call the load() method to:
# 1. Send an HTTP request to the specified URL
# 2. Download the HTML content
# 3. Parse the HTML to extract meaningful text
# 4. Create a list of Document objects containing the extracted content
web_data = loader.load()

# Print the first 1000 characters of the page content from the first Document
# This provides a preview of the successfully loaded web content
# web_data[0] accesses the first Document in the list
# .page_content accesses the text content of that Document
# [:1000] slices the string to get only the first 1000 characters
print(web_data[0].page_content[:1000])

USER_AGENT environment variable not set, consider setting it to identify your requests.


LangChain Overview - Docs by LangChainSkip to main contentLangGraph Platform is now part of LangSmith. Check out the Changelog for more information.Docs by LangChain home pageLangChain + LangGraphSearch...⌘KGitHubTry LangSmithTry LangSmithSearch...NavigationLangChain OverviewLangChainLangGraphIntegrationsLearnReferenceContributingPythonOverviewLangChain v1.0Release notesMigration guideGet startedInstallQuickstartPhilosophyCore componentsAgentsModelsMessagesToolsShort-term memoryStreamingAdvanced usageMiddlewareGuardrailsStructured outputRuntimeContext engineeringModel Context Protocol (MCP)Human-in-the-loopMulti-agentRetrievalLong-term memoryUse in productionStudioTestDeployAgent Chat UIObservabilityOn this page Install Create an agent Core benefitsLangChain OverviewCopy pageCopy pageLangChain v1.0Welcome to the new LangChain documentation! If you encounter any issues or have feedback, please open an issue so we can improve. Archived v0 documentation can be found here.See the release n

#### Text splitters


After you load documents, you will often want to transform those documents to better suit your application.


One of the most simple examples of making documents better suit your application is to split a long document into smaller chunks that can fit into your model's context window. LangChain has built-in document transformers that ease the process of splitting, combining, filtering, and otherwise manipulating documents.

At a high level, here is how text splitters work:

1. They split the text into small, semantically meaningful chunks (often sentences).
2. They start combining these small chunks of text into a larger chunk until you reach a certain size (as measured by a specific function).
3. After the combined text reaches the new chunk's size, make that chunk its own piece of text and then start creating a new chunk of text with some overlap to keep context between chunks.

For a list of types of text splitters LangChain supports, see [LangChain Text Splitters](https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/).


Let's use a simple `CharacterTextSplitter` as an example of how to split the LangChain paper you just loaded.

This is the simplest method. This splits based on characters (by default "\n\n") and measures chunk length by number of characters.

`CharacterTextSplitter` is the simplest method of splitting the content. These splits are based on characters (by default "\n\n") and measures chunk length by number of characters.


In [39]:
# Import the CharacterTextSplitter class from langchain.text_splitter module
# Text splitters are used to divide large texts into smaller, manageable chunks
from langchain.text_splitter import CharacterTextSplitter

# Create a CharacterTextSplitter with specific configuration:
# - chunk_size=200: Each chunk will contain approximately 200 characters
# - chunk_overlap=20: Consecutive chunks will overlap by 20 characters to maintain context
# - separator="\n": Text will be split at newline characters when possible
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=20, separator="\n")

# Split the previously loaded document (PDF or other text) into chunks
# The split_documents method:
# 1. Takes a list of Document objects
# 2. Splits each document's content based on the configured parameters
# 3. Returns a new list of Document objects where each contains a chunk of text
# 4. Preserves the original metadata for each chunk
chunks = text_splitter.split_documents(document)

# Print the total number of chunks created
# This shows how many smaller Document objects were generated from the original document(s)
# The number depends on the original document length and the chunk_size setting
print(len(chunks))

148


The CharacterTextSplitter splits the document into 148 chunks. Let's look at the content of a chunk:


In [40]:
chunks[5].page_content   # take a look at any chunk's page content

'contextualized language models to introduce MindGuide, an \ninnovative chatbot serving as a mental health assistant for \nindividuals seeking guidance and support in these critical areas.'

### Exercise 3
#### Working with Document Loaders and Text Splitters

You now know about about Document objects and how to load content from different sources. Now, let's implement a workflow to load documents, split them, and prepare them for retrieval.

**Instructions:**

1. Import the necessary document loaders to work with both PDF and web content.
2. Load the provided paper about LangChain architecture.
3. Create two different text splitters with varying parameters.
4. Compare the resulting chunks from different splitters.
5. Examine the metadata preservation across splitting.
6. Create a simple function to display statistics about your document chunks.

**Starter code: provide your solution in the TODO parts**


In [41]:
from langchain_core.documents import Document
from langchain_community.document_loaders import PyPDFLoader, WebBaseLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

# Load the LangChain paper
paper_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/96-FDF8f7coh0ooim7NyEQ/langchain-paper.pdf"
pdf_loader = PyPDFLoader(paper_url)
pdf_document = pdf_loader.load()

# Load content from LangChain website
web_url = "https://python.langchain.com/v0.2/docs/introduction/"
web_loader = WebBaseLoader(web_url)
web_document = web_loader.load()

# Create two different text splitters
splitter_1 = CharacterTextSplitter(chunk_size=300, chunk_overlap=30, separator="\n")
splitter_2 = CharacterTextSplitter(chunk_size=800, chunk_overlap=50, separator="\n")  # Create a different splitter with different parameters

# Apply both splitters to the PDF document
chunks_1 = splitter_1.split_documents(pdf_document)
chunks_2 = splitter_2.split_documents(pdf_document)

# Define a function to display document statistics
def display_document_stats(docs, name):
    """Display statistics about a list of document chunks"""
    total_chunks = len(docs)
    total_chars = sum(len(doc.page_content) for doc in docs)
    avg_chunk_size = total_chars / total_chunks if total_chunks > 0 else 0
    
    # Count unique metadata keys across all documents
    all_metadata_keys = set()
    for doc in docs:
        all_metadata_keys.update(doc.metadata.keys())
    
    # Print the statistics
    print(f"\n=== {name} Statistics ===")
    print(f"Total number of chunks: {total_chunks}")
    print(f"Average chunk size: {avg_chunk_size:.2f} characters")
    print(f"Metadata keys preserved: {', '.join(all_metadata_keys)}")
    
    if docs:
        print("\nExample chunk:")
        example_doc = docs[min(5, total_chunks-1)]  # Get the 5th chunk or the last one if fewer
        print(f"Content (first 150 chars): {example_doc.page_content[:150]}...")
        print(f"Metadata: {example_doc.metadata}")
        
        # Calculate length distribution
        lengths = [len(doc.page_content) for doc in docs]
        min_len = min(lengths)
        max_len = max(lengths)
        print(f"Min chunk size: {min_len} characters")
        print(f"Max chunk size: {max_len} characters")

# Display stats for both chunk sets
display_document_stats(chunks_1, "Splitter 1")
display_document_stats(chunks_2, "Splitter 2")


=== Splitter 1 Statistics ===
Total number of chunks: 95
Average chunk size: 266.07 characters
Metadata keys preserved: author, creator, producer, page, moddate, creationdate, title, total_pages, source, page_label

Example chunk:
Content (first 150 chars): comprehensive support within the field of mental health. 
Additionally, the paper discusses the implementation of 
Streamlit to enhance the user ex pe...
Metadata: {'producer': 'PyPDF', 'creator': 'Microsoft Word', 'creationdate': '2023-12-31T03:50:13+00:00', 'author': 'IEEE', 'moddate': '2023-12-31T03:52:06+00:00', 'title': 's8329 final', 'source': 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/96-FDF8f7coh0ooim7NyEQ/langchain-paper.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}
Min chunk size: 65 characters
Max chunk size: 299 characters

=== Splitter 2 Statistics ===
Total number of chunks: 38
Average chunk size: 676.32 characters
Metadata keys preserved: author, creator, producer, page, moddate, crea

<details>
    <summary>Click here for the solution</summary>

```python
from langchain_core.documents import Document
from langchain_community.document_loaders import PyPDFLoader, WebBaseLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

# Load the LangChain paper
paper_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/96-FDF8f7coh0ooim7NyEQ/langchain-paper.pdf"
pdf_loader = PyPDFLoader(paper_url)
pdf_document = pdf_loader.load()

# Load content from LangChain website
web_url = "https://python.langchain.com/v0.2/docs/introduction/"
web_loader = WebBaseLoader(web_url)
web_document = web_loader.load()

# Create two different text splitters
splitter_1 = CharacterTextSplitter(chunk_size=300, chunk_overlap=30, separator="\n")
splitter_2 = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50, separators=["\n\n", "\n", ". ", " ", ""])

# Apply both splitters to the PDF document
chunks_1 = splitter_1.split_documents(pdf_document)
chunks_2 = splitter_2.split_documents(pdf_document)

# Define a function to display document statistics
def display_document_stats(docs, name):
    """Display statistics about a list of document chunks"""
    total_chunks = len(docs)
    total_chars = sum(len(doc.page_content) for doc in docs)
    avg_chunk_size = total_chars / total_chunks if total_chunks > 0 else 0
    
    # Count unique metadata keys across all documents
    all_metadata_keys = set()
    for doc in docs:
        all_metadata_keys.update(doc.metadata.keys())
    
    # Print the statistics
    print(f"\n=== {name} Statistics ===")
    print(f"Total number of chunks: {total_chunks}")
    print(f"Average chunk size: {avg_chunk_size:.2f} characters")
    print(f"Metadata keys preserved: {', '.join(all_metadata_keys)}")
    
    if docs:
        print("\nExample chunk:")
        example_doc = docs[min(5, total_chunks-1)]  # Get the 5th chunk or the last one if fewer
        print(f"Content (first 150 chars): {example_doc.page_content[:150]}...")
        print(f"Metadata: {example_doc.metadata}")
        
        # Calculate length distribution
        lengths = [len(doc.page_content) for doc in docs]
        min_len = min(lengths)
        max_len = max(lengths)
        print(f"Min chunk size: {min_len} characters")
        print(f"Max chunk size: {max_len} characters")

# Display stats for both chunk sets
display_document_stats(chunks_1, "Splitter 1")
display_document_stats(chunks_2, "Splitter 2")
```

</details>


#### Embedding models


Embedding models are specifically designed to interface with text embeddings.

Embeddings generate a vector representation for a specified piece or "chunk" of text.  Embeddings offer the advantage of allowing you to conceptualize text within a vector space. Consequently, you can perform operations such as semantic search, where you identify pieces of text that are most similar within the vector space.


IBM, OpenAI, Hugging Face, and others offer embedding models. Here, you will use the embedding model from IBM's watsonx.ai to work with the text.


In [43]:
from langchain_mistralai import MistralAIEmbeddings

mistral_api_key = os.getenv("MISTRAL_API_KEY")

embeddings = MistralAIEmbeddings(
        model="mistral-embed"
    )

tokenizer.json: 0.00B [00:00, ?B/s]

The following code embeds content in each of the chunks. You can then output the first 5 numbers in the vector representation of the content of the first chunk.


In [44]:
texts = [text.page_content for text in chunks]

embedding_result = embeddings.embed_documents(texts)
embedding_result[0][:5]

[-0.044647216796875,
 0.023529052734375,
 0.0294036865234375,
 0.00887298583984375,
 0.053131103515625]

#### Vector stores


One of the most common ways to store and search over unstructured data is to embed the text data and store the resulting embedding vectors, and then at query time to embed the unstructured query and retrieve the embedding vectors that are 'most similar' to the embedded query. You can use a [vector store](https://python.langchain.com/v0.1/docs/modules/data_connection/vectorstores/) to store embedded data and perform vector search for you.


You can find many vector store options. Here, the code uses `Chroma`.


In [45]:
from langchain.vectorstores import Chroma

Next, have the embedding model perform the embedding process and store the resulting vectors in the Chroma vector database.

**NOTE**: You can safely ignore the warnings related to telemetry events. They are related to ChromaDB's telemetry collection system and do not affect the functionality of your code. Your vector search and similarity operations will work correctly despite these messages.


In [46]:
docsearch = Chroma.from_documents(chunks, embeddings)

Then you can use a similarity search strategy to retrieve the information that is related to your query. The model returns a list of similar or relevant document chunks. Here, you can view the code that prints the contents of the most similar chunk.


In [47]:
query = "Langchain"
docs = docsearch.similarity_search(query)
print(docs[0].page_content)

D. Chain  
Chains  [11] in LangChain refer to the combination of 
multiple components to achieve specific tasks. They provide 
a structured and modular approach to building language


#### Retrievers


A retriever is an interface that returns documents using an unstructured query. Retrievers are more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) them. You can still use vector stores as the backbone of a retriever. Note that other types of retrievers also exist.

Retrievers accept a string `query` as input and return a list of `Documents` as output.

You can view a list of the advanced retrieval types LangChain supports at [https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/](https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/)



A list of advanced retrieval types LangChain could support is available at [https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/](https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/). Let's introduce the `Vector store-backed retriever` and `Parent document retriever` as examples.


##### **Vector store-backed retrievers**


Vector store retrievers are retrievers that use a vector store to retrieve documents. They are a lightweight wrapper around the vector store class to make it conform to the retriever interface. They use the search methods implemented by a vector store, such as similarity search and MMR (Maximum marginal relevance), to query the texts in the vector store.

Now that you have constructed a vector store `docsearch`, you can easily construct a retriever such as seen in the following code.


In [48]:
# Use the docsearch vector store as a retriever
# This converts the vector store into a retriever interface that can fetch relevant documents
retriever = docsearch.as_retriever()

# Invoke the retriever with the query "Langchain"
# This will:
# 1. Convert the query text "Langchain" into an embedding vector
# 2. Perform a similarity search in the vector store using this embedding
# 3. Return the most semantically similar documents to the query
docs = retriever.invoke("Langchain")

# Access the first (most relevant) document from the retrieval results
# This returns the full Document object including:
# - page_content: The text content of the document
# - metadata: Any associated metadata like source, page numbers, etc.
# The returned document is the one most semantically similar to "Langchain"
docs[0]

Document(metadata={'page': 2, 'page_label': '3', 'creationdate': '2023-12-31T03:50:13+00:00', 'author': 'IEEE', 'total_pages': 6, 'title': 's8329 final', 'creator': 'Microsoft Word', 'producer': 'PyPDF', 'moddate': '2023-12-31T03:52:06+00:00', 'source': 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/96-FDF8f7coh0ooim7NyEQ/langchain-paper.pdf'}, page_content='D. Chain  \nChains  [11] in LangChain refer to the combination of \nmultiple components to achieve specific tasks. They provide \na structured and modular approach to building language')

Note that the results are identical to the results you obtained using the similarity search strategy.


##### **Parent document retrievers**


When splitting documents for retrieval, there are often conflicting goals:

- You want small documents so their embeddings can most accurately reflect their meaning. If the documents are too long, then the embeddings can lose meaning.
- You want to have long enough documents to retain the context of each chunk of text.

The `ParentDocumentRetriever` strikes that balance by splitting and storing small chunks of data. During retrieval, this retriever first fetches the small chunks, but then looks up the parent IDs for the data and returns those larger documents.



In [49]:
from langchain.retrievers import ParentDocumentRetriever
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.storage import InMemoryStore

In [51]:
# Set up two different text splitters for a hierarchical splitting approach:

# 1. Parent splitter creates larger chunks (2000 characters)
# This is used to split documents into larger, more contextually complete sections
parent_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=20, separator='\n')

# 2. Child splitter creates smaller chunks (400 characters)
# This is used to split the parent chunks into smaller pieces for more precise retrieval
child_splitter = CharacterTextSplitter(chunk_size=400, chunk_overlap=20, separator='\n')

# Create a Chroma vector store with:
# - A specific collection name "split_parents" for organization
# - The previously configured Watson embeddings function
vectorstore = Chroma(
    collection_name="split_parents", embedding_function=embeddings
)

# Set up an in-memory storage layer for the parent documents
# This will store the larger chunks that provide context, but won't be directly embedded
store = InMemoryStore()

/var/folders/0z/6z_9lrtd6j3_gnlsyv9rb5b00000gn/T/ipykernel_19964/1120337825.py:14: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [52]:
# Create a ParentDocumentRetriever instance that implements hierarchical document retrieval
retriever = ParentDocumentRetriever(
    # The vector store where child document embeddings will be stored and searched
    # This Chroma instance will contain the embeddings for the smaller chunks
    vectorstore=vectorstore,
    
    # The document store where parent documents will be stored
    # These larger chunks won't be embedded but will be retrieved by ID when needed
    docstore=store,
    
    # The splitter used to create small chunks (400 chars) for precise vector search
    # These smaller chunks are embedded and used for similarity matching
    child_splitter=child_splitter,
    
    # The splitter used to create larger chunks (2000 chars) for better context
    # These parent chunks provide more complete information when retrieved
    parent_splitter=parent_splitter,
)

Then, we add documents to the hierarchical retrieval system:


In [53]:
retriever.add_documents(document)

The following code retrieves and counts the number of parent document IDs stored in the document store


In [54]:
len(list(store.yield_keys()))

16

Next, we verify that the underlying vector store still retrieves the small chunks.


In [55]:
sub_docs = vectorstore.similarity_search("Langchain")

In [56]:
print(sub_docs[0].page_content)

II. LANGCHAIN  
LangChain , with its open -source essence, emerges as a 
promising solution, aiming to simplify the complex process of


And then retrieve the relevant large chunk.


In [57]:
retrieved_docs = retriever.invoke("Langchain")

In [58]:
print(retrieved_docs[0].page_content)

difficult thoughts to consi dering ending everything. This shift 
is observable in how people express themselves and 
interact [2]. 
One practical approach to addressing mental illness and 
preventing suicidal ideation is early identification. Recent 
advancements in deep learning have facilitated the 
development of effective early detection methods [3]. A 
notable trend in natural language processing (NLP)  involves 
the use of contextualized pretrained language models  [4], 
which have garnered substantial attention for their 
effectiveness in various text processing tasks.  
This paper delves into the application of these recent 
advancements in pretrained contextualized large language 
models to introduce MindGuide , an innovative chatbot 
designed to function as a mental health assistant for 
individuals in need of guidance and support in these critical 
areas. MindGuide  relies on the capabilities of LangChain  and 
its ChatModels  [5], specifically Chat OpenAI  [6], as the 
fou

##### **RetrievalQA**


Now that you understand how to retrieve information from a document, you might be interested in exploring some more exciting applications. For instance, you could have the Language Model (LLM) read the paper and summarize it for you, or create a QA bot that can answer your questions based on the paper.

Here's an example using LangChain's `RetrievalQA`.


In [59]:
from langchain.chains import RetrievalQA

In [60]:
# Create a RetrievalQA chain by configuring:
qa = RetrievalQA.from_chain_type(
    # The language model to use for generating answers
    llm=llm,
    
    # The chain type "stuff" means all retrieved documents are simply concatenated and passed to the LLM
    chain_type="stuff",
    
    # The retriever component that will fetch relevant documents
    # docsearch.as_retriever() converts the vector store into a retriever interface
    retriever=docsearch.as_retriever(),
    
    # Whether to include the source documents in the response
    # Set to False to return only the generated answer
    return_source_documents=False
)

# Define a query to test the QA system
# This question asks about the main topic of the paper
query = "what is this paper discussing?"

# Execute the QA chain with the query
# This will:
# 1. Send the query to the retriever to get relevant documents
# 2. Combine those documents using the "stuff" method
# 3. Send the query and combined documents to the Llama LLM
# 4. Return the generated answer (without source documents)
qa.invoke(query)

{'query': 'what is this paper discussing?',
 'result': 'This paper appears to be discussing the application of recent advancements in pretrained contextualized large language models, specifically the OpenAI GPT-4 model, in various text processing tasks and mental health support.'}

### Exercise 4
#### **Building a Simple Retrieval System with LangChain**

In this exercise, you'll implement a simple retrieval system using LangChain's vector store and retriever components to help answer questions based on a document.

**Instructions:**

1. Import the necessary components for document loading, embedding, and retrieval.
2. Load the provided document about artificial intelligence.
3. Split the document into manageable chunks.
4. Use an embedding model to create vector representations.
5. Create a vector store and a retriever.
6. Implement a simple question-answering system.
7. Test your system with at least 3 different questions.

**Starter code: provide your solution in the TODO parts**


In [63]:
from langchain_core.documents import Document
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

# 1. Load a document about AI
loader = WebBaseLoader("https://python.langchain.com/v0.2/docs/introduction/")
documents = loader.load()

# 2. Split the document into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

# 4. Create a vector store
vector_store = Chroma.from_documents(chunks, embeddings)

# 5. Create a retriever
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

# 6. Define a function to search for relevant information
def search_documents(query, top_k=3):
    """Search for documents relevant to a query"""
    # Use the retriever to get relevant documents
    docs = retriever.get_relevant_documents(query)
    
    # Limit to top_k if specified
    return docs[:top_k]

# 7. Test with a few queries
test_queries = [
    "What is LangChain?",
    "How do retrievers work?",
    "Why is document splitting important?"
]

for query in test_queries:
    print(f"\nQuery: {query}")
    results = search_documents(query)
    # Print the results
    print(f"Found {len(results)} relevant documents:")
    for i, doc in enumerate(results):
        print(f"\nResult {i+1}: {doc.page_content[:150]}...")
        print(f"Source: {doc.metadata.get('source', 'Unknown')}")




Query: What is LangChain?
Found 3 relevant documents:

Result 1: D. Chain  
Chains  [11] in LangChain refer to the combination of 
multiple components to achieve specific tasks. They provide 
a structured and modula...
Source: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/96-FDF8f7coh0ooim7NyEQ/langchain-paper.pdf

Result 2: LangChain Overview - Docs by LangChainSkip to main contentLangGraph Platform is now part of LangSmith. Check out the Changelog for more information.Do...
Source: https://python.langchain.com/v0.2/docs/introduction/

Result 3: LangChain Overview - Docs by LangChainSkip to main contentLangGraph Platform is now part of LangSmith. Check out the Changelog for more information.Do...
Source: https://python.langchain.com/v0.2/docs/introduction/

Query: How do retrievers work?
Found 3 relevant documents:

Result 1: the user, while others may be retrieve d from memory. In a 
single run, a chain will interact with its memory system twice.  
1. A chain wi

<details>
    <summary>Click here for the solution</summary>

```python
from langchain_core.documents import Document
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_ibm import WatsonxEmbeddings
from ibm_watsonx_ai.metanames import EmbedTextParamsMetaNames
from langchain.chains import RetrievalQA
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM

# 1. Load a document about AI
loader = WebBaseLoader("https://python.langchain.com/v0.2/docs/introduction/")
documents = loader.load()

# 2. Split the document into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

# 3. Set up the embedding model
embed_params = {
    EmbedTextParamsMetaNames.TRUNCATE_INPUT_TOKENS: 3,
    EmbedTextParamsMetaNames.RETURN_OPTIONS: {"input_text": True},
}

embedding_model = WatsonxEmbeddings(
    model_id="ibm/slate-125m-english-rtrvr",
    url="https://us-south.ml.cloud.ibm.com",
    project_id="skills-network",
    params=embed_params,
)

# 4. Create a vector store
vector_store = Chroma.from_documents(chunks, embedding_model)

# 5. Create a retriever
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

# 6. Define a function to search for relevant information
def search_documents(query, top_k=3):
    """Search for documents relevant to a query"""
    # Use the retriever to get relevant documents
    docs = retriever.get_relevant_documents(query)
    
    # Limit to top_k if specified
    return docs[:top_k]

# 7. Test with a few queries
test_queries = [
    "What is LangChain?",
    "How do retrievers work?",
    "Why is document splitting important?"
]

for query in test_queries:
    print(f"\nQuery: {query}")
    results = search_documents(query)
    
    # Print the results
    print(f"Found {len(results)} relevant documents:")
    for i, doc in enumerate(results):
        print(f"\nResult {i+1}: {doc.page_content[:150]}...")
        print(f"Source: {doc.metadata.get('source', 'Unknown')}")
```

</details>


### Memory


Most LLM applications have a conversational interface. An essential component of a conversation is being able to refer to information introduced earlier in the conversation. At a bare minimum, a conversational system should be able to directly access some window of past messages.


#### Chat message history


One of the core utility classes underpinning most (if not all) memory modules is the `ChatMessageHistory` class. This class is a super lightweight wrapper that provides convenience methods for saving `HumanMessages` and `AIMessages`, and then fetching both types of messages.

Here is an example.


In [64]:
# Import the ChatMessageHistory class from langchain.memory
from langchain.memory import ChatMessageHistory

# Set up the language model to use for chat interactions
chat = llm

# Create a new conversation history object
# This will store the back-and-forth messages in the conversation
history = ChatMessageHistory()

# Add an initial greeting message from the AI to the history
# This represents a message that would have been sent by the AI assistant
history.add_ai_message("hi!")

# Add a user's question to the conversation history
# This represents a message sent by the user
history.add_user_message("what is the capital of France?")

Let's have a look at the messages in the history:


In [65]:
history.messages

[AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is the capital of France?', additional_kwargs={}, response_metadata={})]

You can pass these messages in history to the model to generate a response. The code below is retrieving all messages from the ChatMessageHistory object and passing them to the Llama LLM to generate a contextually appropriate response based on the conversation history.


In [66]:
ai_response = chat.invoke(history.messages)
ai_response

AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 49, 'total_tokens': 57, 'completion_time': 0.005934782, 'prompt_time': 0.002886102, 'queue_time': 0.168011605, 'total_time': 0.008820884}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_75db6866d3', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--1300253d-a441-48e1-8404-a58262577ae5-0', usage_metadata={'input_tokens': 49, 'output_tokens': 8, 'total_tokens': 57})

You can see the model gives a correct response.


Let's look again at the messages in history. Note that the history now includes the AI's message, which has been appended to the message history:


In [67]:
history.add_ai_message(ai_response)
history.messages

[AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is the capital of France?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 49, 'total_tokens': 57, 'completion_time': 0.005934782, 'prompt_time': 0.002886102, 'queue_time': 0.168011605, 'total_time': 0.008820884}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_75db6866d3', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--1300253d-a441-48e1-8404-a58262577ae5-0', usage_metadata={'input_tokens': 49, 'output_tokens': 8, 'total_tokens': 57})]

#### Conversation buffer


Conversation buffer memory allows for the storage of messages, which you use to extract messages to a variable. Consider using conversation buffer memory in a chain, setting `verbose=True` so that the prompt is visible.


In [68]:
# Import ConversationBufferMemory from langchain.memory module
from langchain.memory import ConversationBufferMemory

# Import ConversationChain from langchain.chains module
from langchain.chains import ConversationChain

# Create a conversation chain with the following components:
conversation = ConversationChain(
    # The language model to use for generating responses
    llm=llm,
    
    # Set verbose to True to see the full prompt sent to the LLM, including memory contents
    verbose=True,
    
    # Initialize with ConversationBufferMemory that will:
    # - Store all conversation turns (user inputs and AI responses)
    # - Append the entire conversation history to each new prompt
    # - Provide context for the LLM to generate contextually relevant responses
    memory=ConversationBufferMemory()
)

/var/folders/0z/6z_9lrtd6j3_gnlsyv9rb5b00000gn/T/ipykernel_19964/3775372964.py:19: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationBufferMemory()
/var/folders/0z/6z_9lrtd6j3_gnlsyv9rb5b00000gn/T/ipykernel_19964/3775372964.py:8: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  conversation = ConversationChain(


Let’s begin the conversation by introducing the user as a little cat and proceed by incorporating some additional messages. Finally, prompt the model to check if it can recall that the user is a little cat.


In [69]:
conversation.invoke(input="Hello, I am a little cat. Who are you?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hello, I am a little cat. Who are you?
AI:

> Finished chain.


{'input': 'Hello, I am a little cat. Who are you?',
 'history': '',
 'response': 'Hello there little cat. I\'m an artificial intelligence, a type of computer program designed to simulate human-like conversations and answer questions to the best of my knowledge. I\'m often referred to as a conversational AI or chatbot. I don\'t have a physical body, but I exist in the digital realm and can be accessed through various devices connected to the internet. I\'m here to help answer any questions you may have, so feel free to ask me anything. By the way, I see you\'re referring to yourself as a "little cat." Are you perhaps thinking of the phrase "little black cat" from the classic novel "Alice\'s Adventures in Wonderland" by Lewis Carroll?'}

In [70]:
conversation.invoke(input="What can you do?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello, I am a little cat. Who are you?
AI: Hello there little cat. I'm an artificial intelligence, a type of computer program designed to simulate human-like conversations and answer questions to the best of my knowledge. I'm often referred to as a conversational AI or chatbot. I don't have a physical body, but I exist in the digital realm and can be accessed through various devices connected to the internet. I'm here to help answer any questions you may have, so feel free to ask me anything. By the way, I see you're referring to yourself as a "little cat." Are you perhaps thinking of the phrase "little black cat" from the classic novel "Alice's Adventures in Wo

{'input': 'What can you do?',
 'history': 'Human: Hello, I am a little cat. Who are you?\nAI: Hello there little cat. I\'m an artificial intelligence, a type of computer program designed to simulate human-like conversations and answer questions to the best of my knowledge. I\'m often referred to as a conversational AI or chatbot. I don\'t have a physical body, but I exist in the digital realm and can be accessed through various devices connected to the internet. I\'m here to help answer any questions you may have, so feel free to ask me anything. By the way, I see you\'re referring to yourself as a "little cat." Are you perhaps thinking of the phrase "little black cat" from the classic novel "Alice\'s Adventures in Wonderland" by Lewis Carroll?',
 'response': "I'm glad you asked. I can do a wide range of things, little cat. I can answer questions on various topics, from science and history to entertainment and culture. I can provide definitions for words, explain concepts, and even off

In [71]:
conversation.invoke(input="Who am I?.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello, I am a little cat. Who are you?
AI: Hello there little cat. I'm an artificial intelligence, a type of computer program designed to simulate human-like conversations and answer questions to the best of my knowledge. I'm often referred to as a conversational AI or chatbot. I don't have a physical body, but I exist in the digital realm and can be accessed through various devices connected to the internet. I'm here to help answer any questions you may have, so feel free to ask me anything. By the way, I see you're referring to yourself as a "little cat." Are you perhaps thinking of the phrase "little black cat" from the classic novel "Alice's Adventures in Wo

{'input': 'Who am I?.',
 'history': 'Human: Hello, I am a little cat. Who are you?\nAI: Hello there little cat. I\'m an artificial intelligence, a type of computer program designed to simulate human-like conversations and answer questions to the best of my knowledge. I\'m often referred to as a conversational AI or chatbot. I don\'t have a physical body, but I exist in the digital realm and can be accessed through various devices connected to the internet. I\'m here to help answer any questions you may have, so feel free to ask me anything. By the way, I see you\'re referring to yourself as a "little cat." Are you perhaps thinking of the phrase "little black cat" from the classic novel "Alice\'s Adventures in Wonderland" by Lewis Carroll?\nHuman: What can you do?\nAI: I\'m glad you asked. I can do a wide range of things, little cat. I can answer questions on various topics, from science and history to entertainment and culture. I can provide definitions for words, explain concepts, and

As you can see, the model remembers that the user is a little cat. You can see this in both the `history` and the `response` keys in the dictionary returned by the `conversation.invoke()` method.


### Exercise 5
#### **Building a Chatbot with Memory using LangChain**

In this exercise, you'll create a simple chatbot that can remember previous interactions using LangChain's memory components. You'll implement conversation memory to make your chatbot maintain context throughout a conversation.

**Instructions:**

1. Import the necessary components for chat history and conversation memory.
2. Set up a language model for your chatbot.
3. Create a conversation chain with memory capabilities.
4. Implement a simple interactive chat interface.
5. Test the memory capabilities with a series of related questions.
6. Examine how the conversation history is stored and accessed.
**Starter code: provide your solution in the TODO parts**


In [72]:
from langchain.memory import ConversationBufferMemory, ChatMessageHistory
from langchain.chains import ConversationChain
from langchain_core.messages import HumanMessage, AIMessage


# 2. Create a simple conversation with chat history
history = ChatMessageHistory()

# Add some initial messages (optional)
history.add_user_message("Hello, my name is Alice.")
history.add_ai_message("Hello Alice, I am AI at your service")

# 3. Print the current conversation history
print(history.messages)

# 4. Set up a conversation chain with memory
memory = ConversationBufferMemory()
conversation = ConversationChain(llm=llm,
                                 memory=memory)

# 5. Function to simulate a conversation
def chat_simulation(conversation, inputs):
    """Run a series of inputs through the conversation chain and display responses"""
    print("\n=== Beginning Chat Simulation ===")
    
    for i, user_input in enumerate(inputs):
        print(f"\n--- Turn {i+1} ---")
        print(f"Human: {user_input}")
        
        # Get response from the conversation chain
        response = conversation.invoke(input=user_input)
        
        # Print the AI's response
        print(f"AI: {response['response']}")
    
    print("\n=== End of Chat Simulation ===")

# 6. Test with a series of related questions
test_inputs = [
    "My favorite color is blue.",
    "I enjoy hiking in the mountains.",
    "What activities would you recommend for me?",
    "What was my favorite color again?",
    "Can you remember both my name and my favorite color?"
]

chat_simulation(conversation, test_inputs)

# 7. Examine the conversation memory
print("\nFinal Memory Contents:")
print(conversation.memory.buffer)

# 8. Create a new conversation with a different type of memory (optional)
# Try implementing ConversationSummaryMemory or another type of memory

[HumanMessage(content='Hello, my name is Alice.', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello Alice, I am AI at your service', additional_kwargs={}, response_metadata={})]

=== Beginning Chat Simulation ===

--- Turn 1 ---
Human: My favorite color is blue.
AI: Blue is a wonderful color, often associated with feelings of calmness and serenity. Did you know that blue is also the favorite color of around 42% of people in the United States, according to a survey conducted by YouGov in 2015? It's also the color of the sky and the ocean, which might be why it's often linked to nature and tranquility. What do you like most about the color blue?

--- Turn 2 ---
Human: I enjoy hiking in the mountains.
AI: That sounds like a wonderful hobby. Hiking in the mountains can be very invigorating and offers breathtaking views. Did you know that the highest mountain in the world, Mount Everest, stands at an impressive 8,848 meters (29,029 feet) above sea level? It's located in 

<details>
    <summary>Click here for the solution</summary>

```python
from langchain.memory import ConversationBufferMemory, ChatMessageHistory
from langchain.chains import ConversationChain
from langchain_core.messages import HumanMessage, AIMessage
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM

# 1. Set up the language model
model_id = 'meta-llama/llama-4-maverick-17b-128e-instruct-fp8'
parameters = {
    GenParams.MAX_NEW_TOKENS: 256,
    GenParams.TEMPERATURE: 0.2,
}
credentials = {"url": "https://us-south.ml.cloud.ibm.com"}
project_id = "skills-network"

# Initialize the model
model = ModelInference(
    model_id=model_id,
    params=parameters,
    credentials=credentials,
    project_id=project_id
)
llm = WatsonxLLM(model=model)

# 2. Create a simple conversation with chat history
history = ChatMessageHistory()

# Add some initial messages
history.add_user_message("Hello, my name is Alice.")
history.add_ai_message("Hello Alice! It's nice to meet you. How can I help you today?")

# 3. Print the current conversation history
print("Initial Chat History:")
for message in history.messages:
    sender = "Human" if isinstance(message, HumanMessage) else "AI"
    print(f"{sender}: {message.content}")

# 4. Set up a conversation chain with memory
memory = ConversationBufferMemory(chat_memory=history)
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

# 5. Function to simulate a conversation
def chat_simulation(conversation, inputs):
    """Run a series of inputs through the conversation chain and display responses"""
    print("\n=== Beginning Chat Simulation ===")
    
    for i, user_input in enumerate(inputs):
        print(f"\n--- Turn {i+1} ---")
        print(f"Human: {user_input}")
        
        # Get response from the conversation chain
        response = conversation.invoke(input=user_input)
        
        # Print the AI's response
        print(f"AI: {response['response']}")
    
    print("\n=== End of Chat Simulation ===")

# 6. Test with a series of related questions
test_inputs = [
    "My favorite color is blue.",
    "I enjoy hiking in the mountains.",
    "What activities would you recommend for me?",
    "What was my favorite color again?",
    "Can you remember both my name and my favorite color?"
]

chat_simulation(conversation, test_inputs)

# 7. Examine the conversation memory
print("\nFinal Memory Contents:")
print(conversation.memory.buffer)

# 8. Create a new conversation with a different type of memory (optional)
from langchain.memory import ConversationSummaryMemory

# Create a summarizing memory that will compress the conversation
summary_memory = ConversationSummaryMemory(llm=llm)
# Save the initial context to the summary memory
summary_memory.save_context(
    {"input": "Hello, my name is Alice."}, 
    {"output": "Hello Alice! It's nice to meet you. How can I help you today?"}
)
summary_conversation = ConversationChain(
   llm=llm,
   memory=summary_memory,
   verbose=True
)

print("\\\\n\\n=== Testing Conversation Summary Memory ===")
# Let's use the same inputs for comparison
chat_simulation(summary_conversation, test_inputs)

print("\\nFinal Summary Memory Contents:")
print(summary_memory.buffer)

# 9. Compare the two memory types
print("\n=== Memory Comparison ===")
print(f"Buffer Memory Size: {len(conversation.memory.buffer)} characters")
print(f"Summary Memory Size: {len(summary_memory.buffer)} characters")
print("\nThe conversation summary memory typically creates a more compact representation of the chat history.")
```

</details>


### Chains


`Chains` are one of the most powerful features in LangChain, allowing you to combine multiple components into cohesive workflows. This section presents two different methodologies for implementing chains - the traditional `SequentialChain` approach and the newer LangChain Expression Language (`LCEL`).

**Why Chains Matter:**

Chains solve a fundamental problem with LLMs. Chains are primarily designed to handle a single prompt and generate a single response. However, most real-world applications require multi-step reasoning, accessing different tools, or breaking complex tasks into manageable pieces. Chains allow you to orchestrate these complex workflows.

**Evolution of Chain Patterns:**

Traditional chains (`LLMChain`, `SequentialChain`) were LangChain's first implementation, offering a structured but somewhat rigid approach. LCEL (using the pipe operator `|`) represents a more flexible, functional approach that's easier to compose and debug.

**Note:** While both approaches are presented here for educational purposes, **LCEL is the recommended pattern for new development.** The SequentialChain approach continues to be supported for backward compatibility, but the LangChain community has largely transitioned to the LCEL pattern for its superior flexibility and expressiveness.


#### **Simple Chain**


#### Traditional Approach: LLMChain


Here is a simple single chain using `LLMChain`.


In [80]:
# Import the LLMChain class from langchain.chains module
from langchain.chains import LLMChain

# Create a template string for generating recommendations of classic dishes from a given location
# The template includes:
# - Instructions for the task (recommending a classic dish)
# - A placeholder {location} that will be replaced with user input
# - A format indicator for the expected response
template = """Your job is to come up with a classic dish from the area that the users suggests.
{location}
 YOUR RESPONSE:
"""

# Create a PromptTemplate object by providing:
# - The template string defined above
# - A list of input variables that will be used to format the template
prompt_template = PromptTemplate(template=template, input_variables=['location'])

# Create an LLMChain that connects:
# - The Llama language model (llama_llm)
# - The prompt template configured for location-based dish recommendations
# - An output_key 'meal' that specifies the key name for the chain's response in the output dictionary
location_chain = LLMChain(llm=llm, prompt=prompt_template, output_key='meal')

# Invoke the chain with 'China' as the location input
# This will:
# 1. Format the template with {location: 'China'}
# 2. Send the formatted prompt to the Llama LLM
# 3. Return a dictionary with the response under the key 'meal'
location_chain.invoke(input={'location':'China'})

{'location': 'China',
 'meal': "For China, I'd recommend a classic dish that's both iconic and delicious: \n\n**Kung Pao Chicken**\n\nThis spicy Sichuan dish originated in the 1950s and has since become a staple of Chinese cuisine worldwide. It's made with marinated chicken, peanuts, vegetables, and chili peppers in a savory sauce, all stir-fried to perfection. The combination of flavors and textures is a true representation of Chinese cooking, and it's a must-try for anyone looking to experience the authentic taste of China.\n\nWould you like to know more about the history or ingredients of Kung Pao Chicken?"}

#### Modern Approach: LCEL

Here is the same chain implemented using the more modern LCEL (LangChain Expression Language) approach with the pipe operator:


In [76]:
# Import PromptTemplate from langchain_core.prompts
# This is the new import path in LangChain's modular structure
from langchain_core.prompts import PromptTemplate

# Import StrOutputParser from langchain_core.output_parsers
from langchain_core.output_parsers import StrOutputParser

template = """Your job is to come up with a classic dish from the area that the users suggests.
{location}
 YOUR RESPONSE:
"""

# Create a prompt template using the from_template method
prompt = PromptTemplate.from_template(template)

# Create a chain using LangChain Expression Language (LCEL) with the pipe operator
# This creates a processing pipeline that:
# 1. Formats the prompt with the input values
# 2. Sends the formatted prompt to the Llama LLM
# 3. Parses the output to extract just the string response
location_chain_lcel = prompt | llm | StrOutputParser()

# Invoke the chain with 'China' as the location
result = location_chain_lcel.invoke({"location": "China"})

# Print the result (the recommended classic dish from China)
print(result)

For China, I'd recommend a classic dish that's both iconic and delicious: 

**Kung Pao Chicken**

This spicy Sichuan dish originated in the 1950s and has since become a staple of Chinese cuisine worldwide. It's made with marinated chicken, peanuts, vegetables, and chili peppers in a savory sauce, all stir-fried to perfection. The combination of flavors and textures is a true representation of Chinese cooking, and it's a must-try for anyone looking to experience the authentic taste of China.

Would you like to know more about the history or ingredients of Kung Pao Chicken?


#### **Simple sequential chain**


Sequential chains allow you to use output of one LLM as the input for another LLM. This approach is beneficial for dividing tasks and maintaining the focus of your LLM.

In this example, you see a sequence that:

- Gets a meal from a location
- Gets a recipe for that meal
- Estimates the cooking time for that recipe

This pattern is incredibly valuable for breaking down complex tasks into logical steps, where each step depends on the output of the previous step. The traditional approach uses `SequentialChain`, while the modern `LCEL` approach uses piping and `RunnablePassthrough.assign`.


#### Traditional Approach: `SequentialChain`


In [77]:
# Import SequentialChain from langchain.chains module
from langchain.chains import SequentialChain

# Create a template for generating a recipe based on a meal
template = """Given a meal {meal}, give a short and simple recipe on how to make that dish at home.
 YOUR RESPONSE:
"""

# Create a PromptTemplate with 'meal' as the input variable
prompt_template = PromptTemplate(template=template, input_variables=['meal'])

# Create an LLMChain (chain 2) for generating recipes
# The output_key='recipe' defines how this chain's output will be referenced in later chains
dish_chain = LLMChain(llm=llm, prompt=prompt_template, output_key='recipe')

In [78]:
# Create a template for estimating cooking time based on a recipe
# This template asks the LLM to analyze a recipe and estimate preparation time
template = """Given the recipe {recipe}, estimate how much time I need to cook it.
 YOUR RESPONSE:
"""

# Create a PromptTemplate with 'recipe' as the input variable
prompt_template = PromptTemplate(template=template, input_variables=['recipe'])

# Create an LLMChain (chain 3) for estimating cooking time
# The output_key='time' defines the key for this chain's output in the final result
recipe_chain = LLMChain(llm=llm, prompt=prompt_template, output_key='time')

In [81]:
# Create a SequentialChain that combines all three chains:
# 1. location_chain (from earlier code): Takes a location and suggests a dish
# 2. dish_chain: Takes the suggested dish and provides a recipe
# 3. recipe_chain: Takes the recipe and estimates cooking time
overall_chain = SequentialChain(
    # List of chains to execute in sequence
    chains=[location_chain, dish_chain, recipe_chain],
    
    # The input variables required to start the chain sequence
    # Only 'location' is needed to begin the process
    input_variables=['location'],
    
    # The output variables to include in the final result
    # This makes the output of each chain available in the final result
    output_variables=['meal', 'recipe', 'time'],
    
    # Whether to print detailed information about each step
    verbose=True
)

Let's use ```pprint``` to print the response to make it more clear.


In [82]:
from pprint import pprint
pprint(overall_chain.invoke(input={'location':'China'}))



> Entering new SequentialChain chain...

> Finished chain.
{'location': 'China',
 'meal': "For China, I'd recommend a classic dish that's both iconic and "
         'delicious: \n'
         '\n'
         '**Kung Pao Chicken**\n'
         '\n'
         'This spicy Sichuan dish originated in the 1950s and has since become '
         "a staple of Chinese cuisine worldwide. It's made with marinated "
         'chicken, peanuts, vegetables, and chili peppers in a savory sauce, '
         'all stir-fried to perfection. The combination of flavors and '
         "textures is a true representation of Chinese cooking, and it's a "
         'must-try for anyone looking to experience the authentic taste of '
         'China.\n'
         '\n'
         'Would you like to know more about the history or ingredients of Kung '
         'Pao Chicken?',
 'recipe': 'Kung Pao Chicken is indeed a beloved dish around the world, and '
           "for good reason. It's a perfect blend of spicy, savory, and sw

#### Modern Approach: LCEL 

Here is the same sequential chain implemented using the modern LCEL approach:


In [83]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Define the templates for each step
location_template = """Your job is to come up with a classic dish from the area that the users suggests.
{location}

YOUR RESPONSE:
"""

dish_template = """Given a meal {meal}, give a short and simple recipe on how to make that dish at home.

YOUR RESPONSE:
"""

time_template = """Given the recipe {recipe}, estimate how much time I need to cook it.

YOUR RESPONSE:
"""

# Create the location chain using LCEL (LangChain Expression Language)
# This chain takes a location and returns a classic dish from that region
location_chain_lcel = (
    PromptTemplate.from_template(location_template)  # Format the prompt with location
    | llm                                    # Send to the LLM
    | StrOutputParser()                              # Extract the string response
)

# Create the dish chain using LCEL
# This chain takes a meal name and returns a recipe
dish_chain_lcel = (
    PromptTemplate.from_template(dish_template)      # Format the prompt with meal
    | llm                                    # Send to the LLM
    | StrOutputParser()                              # Extract the string response
)

# Create the time estimation chain using LCEL
# This chain takes a recipe and returns an estimated cooking time
time_chain_lcel = (
    PromptTemplate.from_template(time_template)      # Format the prompt with recipe
    | llm                                    # Send to the LLM
    | StrOutputParser()                              # Extract the string response
)

# Combine all chains into a single workflow using RunnablePassthrough.assign
# RunnablePassthrough.assign adds new keys to the input dictionary without removing existing ones
overall_chain_lcel = (
    # Step 1: Generate a meal based on location and add it to the input dictionary
    RunnablePassthrough.assign(meal=lambda x: location_chain_lcel.invoke({"location": x["location"]}))
    # Step 2: Generate a recipe based on the meal and add it to the input dictionary
    | RunnablePassthrough.assign(recipe=lambda x: dish_chain_lcel.invoke({"meal": x["meal"]}))
    # Step 3: Estimate cooking time based on the recipe and add it to the input dictionary
    | RunnablePassthrough.assign(time=lambda x: time_chain_lcel.invoke({"recipe": x["recipe"]}))
)
# Run the chain
result = overall_chain_lcel.invoke({"location": "China"})
pprint(result)

{'location': 'China',
 'meal': "For China, I'd like to suggest a classic dish that's both iconic and "
         'delicious: \n'
         '\n'
         '**Kung Pao Chicken**\n'
         '\n'
         'This spicy Sichuan dish originated in the 1950s and has since become '
         "a staple of Chinese cuisine worldwide. It's made with marinated "
         'chicken, peanuts, vegetables, and chili peppers in a savory sauce, '
         'all stir-fried to perfection. The combination of flavors and '
         "textures is a true representation of Chinese cooking, and it's a "
         'must-try for anyone looking to experience the authentic taste of '
         'China.\n'
         '\n'
         'Would you like to know more about the history or ingredients of Kung '
         'Pao Chicken?',
 'recipe': 'Kung Pao Chicken is indeed a classic and iconic dish that has '
           "captured the hearts of many around the world. I'd love to know "
           'more about its history and ingredients.\n'

### Exercise 6
#### **Implementing Multi-Step Processing with Different Chain Approaches**

In this exercise, you'll create a multi-step information processing system using both traditional chains and the modern LCEL approach. You'll build a system that analyzes product reviews, extracts key information, and generates responses based on the analysis.

**Instructions:**

1. Import the necessary components for both traditional chains and LCEL.
2. Implement a three-step process using both traditional SequentialChain and modern LCEL approaches.
3. Create templates for sentiment analysis, summarization, and response generation.
4. Test your implementations with sample product reviews.
5. Compare the flexibility and readability of both approaches.
6. Document the advantages and disadvantages of each method.

**Starter code: provide your solution in the TODO parts**


In [93]:
from langchain.chains import LLMChain, SequentialChain
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Sample product reviews for testing
positive_review = """I absolutely love this coffee maker! It brews quickly and the coffee tastes amazing. 
The built-in grinder saves me so much time in the morning, and the programmable timer means 
I wake up to fresh coffee every day. Worth every penny and highly recommended to any coffee enthusiast."""

negative_review = """Disappointed with this laptop. It's constantly overheating after just 30 minutes of use, 
and the battery life is nowhere near the 8 hours advertised - I barely get 3 hours. 
The keyboard has already started sticking on several keys after just two weeks. Would not recommend to anyone."""

# Step 1: Define the prompt templates for each processing step
sentiment_template = """Analyze the sentiment of the following product review as positive, negative, or neutral.
Provide your analysis in the format: "SENTIMENT: [positive/negative/neutral]"

Review: {review}

Your analysis:
"""

summary_template = """Summarize the following product review into 3-5 key bullet points.
Each bullet point should be concise and capture an important aspect mentioned in the review.

Review: {review}
Sentiment: {sentiment}

Key points:
"""

response_template = """Write a helpful response to a customer based on their product review.
If the sentiment is positive, thank them for their feedback. If negative, express understanding 
and suggest a solution or next steps. Personalize based on the specific points they mentioned.

Review: {review}
Sentiment: {sentiment}
Key points: {summary}

Response to customer:
"""

sentiment_prompt = PromptTemplate.from_template(sentiment_template)
summary_prompt = PromptTemplate.from_template(summary_template)
response_prompt = PromptTemplate.from_template(response_template)

# PART 1: Traditional Chain Approach
sentiment_chain = LLMChain(llm=llm, prompt=sentiment_prompt, output_key='sentiment')

summary_chain = LLMChain(llm=llm, prompt=summary_prompt, output_key='summary')

report_chain = LLMChain(llm=llm, prompt=response_prompt, output_key='response')


chain = SequentialChain(
    # List of chains to execute in sequence
    chains=[sentiment_chain, summary_chain, report_chain],
    
    # The input variables required to start the chain sequence
    # Only 'location' is needed to begin the process
    input_variables=['review'],
    
    # The output variables to include in the final result
    # This makes the output of each chain available in the final result
    output_variables=['sentiment', 'summary', 'response'],
    
    # Whether to print detailed information about each step
    verbose=False
)

# PART 2: LCEL Approach
sentiment_chain_lcel = sentiment_prompt|llm|StrOutputParser()
summary_chain_lcel = summary_prompt|llm|StrOutputParser()
report_chain_lcel = response_prompt|llm|StrOutputParser()

lcel_chain = (
    RunnablePassthrough.assign(
        sentiment=lambda x: sentiment_chain_lcel.invoke({"review": x["review"]})
    )
    | RunnablePassthrough.assign(
        summary=lambda x: summary_chain_lcel.invoke({
            "review": x["review"], 
            "sentiment": x["sentiment"]
        })
    )
    | RunnablePassthrough.assign(
        response=lambda x: report_chain_lcel.invoke({
            "review": x["review"], 
            "sentiment": x["sentiment"], 
            "summary": x["summary"]
        })
    )
)

# Test both implementations
def test_chains(review):
    """Test both chain implementations with the given review"""
    print("\n" + "="*50)
    print(f"TESTING WITH REVIEW:\n{review[:100]}...\n")
    
    print("TRADITIONAL CHAIN RESULTS:")
    # TODO: Run the traditional chain and print the results
    response_traditional = chain.invoke(input={'review':review})
    print('sentiment: ',response_traditional['sentiment'])
    print('summary: ',response_traditional['summary'])
    print('response to customer: ',response_traditional['response'])
    print("="*50)

    print("\nLCEL CHAIN RESULTS:")
    # TODO: Run the LCEL chain and print the results
    response_lcel = lcel_chain.invoke({'review':review})
    print('sentiment: ',response_lcel['sentiment'])
    print('summary: ',response_lcel['summary'])
    print('response to customer: ',response_lcel['response'])
    
    print("="*50)

# Run tests
test_chains(positive_review)
test_chains(negative_review)


TESTING WITH REVIEW:
I absolutely love this coffee maker! It brews quickly and the coffee tastes amazing. 
The built-in g...

TRADITIONAL CHAIN RESULTS:
sentiment:  SENTIMENT: positive

The review contains several positive phrases and statements, such as "I absolutely love this coffee maker," "brews quickly," "coffee tastes amazing," "worth every penny," and "highly recommended." These phrases indicate a strong positive sentiment towards the product.
summary:  Here are 3-5 key bullet points summarizing the review:

- The coffee maker brews coffee quickly.
- The built-in grinder saves time in the morning.
- The programmable timer allows for fresh coffee every day.
- The coffee tastes amazing.
- The product is worth every penny and highly recommended.
response to customer:  Dear [Customer's Name],

We're thrilled to hear that you're absolutely loving your coffee maker! We're glad to know that it's meeting your expectations and making your mornings easier and more enjoyable. The built-in

<details>
    <summary>Click here for the solution</summary>
    
```python
from langchain.chains import LLMChain, SequentialChain
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Sample product reviews for testing
positive_review = """I absolutely love this coffee maker! It brews quickly and the coffee tastes amazing. 
The built-in grinder saves me so much time in the morning, and the programmable timer means 
I wake up to fresh coffee every day. Worth every penny and highly recommended to any coffee enthusiast."""

negative_review = """Disappointed with this laptop. It's constantly overheating after just 30 minutes of use, 
and the battery life is nowhere near the 8 hours advertised - I barely get 3 hours. 
The keyboard has already started sticking on several keys after just two weeks. Would not recommend to anyone."""

# Step 1: Define the prompt templates for each processing step
sentiment_template = """Analyze the sentiment of the following product review as positive, negative, or neutral.
Provide your analysis in the format: "SENTIMENT: [positive/negative/neutral]"

Review: {review}

Your analysis:
"""

summary_template = """Summarize the following product review into 3-5 key bullet points.
Each bullet point should be concise and capture an important aspect mentioned in the review.

Review: {review}
Sentiment: {sentiment}

Key points:
"""

response_template = """Write a helpful response to a customer based on their product review.
If the sentiment is positive, thank them for their feedback. If negative, express understanding 
and suggest a solution or next steps. Personalize based on the specific points they mentioned.

Review: {review}
Sentiment: {sentiment}
Key points: {summary}

Response to customer:
"""

# Create prompt templates for each step
sentiment_prompt = PromptTemplate.from_template(sentiment_template)
summary_prompt = PromptTemplate.from_template(summary_template)
response_prompt = PromptTemplate.from_template(response_template)


# PART 1: Traditional Chain Approach
# Create individual LLMChains for each step
sentiment_chain = LLMChain(
    llm=llama_llm, 
    prompt=sentiment_prompt, 
    output_key="sentiment"
)

summary_chain = LLMChain(
    llm=llama_llm, 
    prompt=summary_prompt, 
    output_key="summary"
)

response_chain = LLMChain(
    llm=llama_llm, 
    prompt=response_prompt, 
    output_key="response"
)

# Create a SequentialChain to connect all steps
traditional_chain = SequentialChain(
    chains=[sentiment_chain, summary_chain, response_chain],
    input_variables=["review"],
    output_variables=["sentiment", "summary", "response"],
    verbose=True
)


# PART 2: LCEL Approach
# Create individual chain components using the pipe operator (|)
sentiment_chain_lcel = sentiment_prompt | llama_llm | StrOutputParser()
summary_chain_lcel = summary_prompt | llama_llm | StrOutputParser()
response_chain_lcel = response_prompt | llama_llm | StrOutputParser()

# Connect the components using RunnablePassthrough.assign()
lcel_chain = (
    RunnablePassthrough.assign(
        sentiment=lambda x: sentiment_chain_lcel.invoke({"review": x["review"]})
    )
    | RunnablePassthrough.assign(
        summary=lambda x: summary_chain_lcel.invoke({
            "review": x["review"], 
            "sentiment": x["sentiment"]
        })
    )
    | RunnablePassthrough.assign(
        response=lambda x: response_chain_lcel.invoke({
            "review": x["review"], 
            "sentiment": x["sentiment"], 
            "summary": x["summary"]
        })
    )
)


# Test both implementations
def test_chains(review):
    """Test both chain implementations with the given review"""
    print("\n" + "="*50)
    print(f"TESTING WITH REVIEW:\n{review[:100]}...\n")
    
    print("TRADITIONAL CHAIN RESULTS:")
    traditional_results = traditional_chain.invoke({"review": review})
    print(f"Sentiment: {traditional_results['sentiment']}")
    print(f"Summary: {traditional_results['summary']}")
    print(f"Response: {traditional_results['response']}")
    
    print("\nLCEL CHAIN RESULTS:")
    lcel_results = lcel_chain.invoke({"review": review})
    print(f"Sentiment: {lcel_results['sentiment']}")
    print(f"Summary: {lcel_results['summary']}")
    print(f"Response: {lcel_results['response']}")
    
    print("="*50)

# Run tests
test_chains(positive_review)
test_chains(negative_review)
```
</detail>


### Tools and Agents


##### **Tools**


Tools extend an LLM's capabilities beyond just generating text. They allow the model to actually perform actions in the world or access external systems. This notebook shows the Python REPL tool, but there are many other tools:

- Search tools: Connect to search engines, database queries, or vector stores.
- API tools: Make calls to external web services.
- Human-in-the-loop tools: Request human input for critical decisions.


You can find a list of tools that LangChain supports at [https://python.langchain.com/docs/how_to/#tools](https://python.langchain.com/docs/how_to/#tools).


Let’s explore how to work with tools, using the `Python REPL` tool as an example. The `Python REPL` tool can run Python commands. These commands can either come from the user or the LLM can generate the commands. This tool is particularly useful for complex calculations. Instead of having the LLM generate the answer directly, using the LLM to generate code to calculate the answer is more efficient.


In [94]:
from langchain_core.tools import Tool
from langchain.tools import tool
from langchain_experimental.utilities import PythonREPL

The `@tool` decorator is a convenient way to define tools, but you can also use the Tool class directly:


In [95]:
# Create a PythonREPL instance
# This provides an environment where Python code can be executed as strings
python_repl = PythonREPL()

# Create a Tool using the Tool class
# This wraps the Python REPL functionality as a tool that can be used by agents
python_calculator = Tool(
    # The name of the tool - this helps agents identify when to use this tool
    name="Python Calculator",
    
    # The function that will be called when the tool is used
    # python_repl.run takes a string of Python code and executes it
    func=python_repl.run,
    
    # A description of what the tool does and how to use it
    # This helps the agent understand when and how to use this tool
    description="Useful for when you need to perform calculations or execute Python code. Input should be valid Python code."
)

Let's test this tool with a simple Python command:


In [96]:
python_calculator.invoke("a = 3; b = 1; print(a+b)")

Python REPL can execute arbitrary code. Use with caution.


'4\n'

We can also create custom tools using the `@tool` decorator:


In [97]:
@tool
def search_weather(location: str):
    """Search for the current weather in the specified location."""
    # In a real application, this would call a weather API
    return f"The weather in {location} is currently sunny and 72°F."

##### **Toolkits**


Toolkits are collections of tools that are designed to be used together for specific tasks.

Let's create a simple toolkit that contains multiple tools:


In [98]:
# Create a toolkit (collection of tools)
tools = [python_calculator, search_weather]

A list of toolkits that Langchain supports is available at [https://python.langchain.com/docs/concepts/tools/#toolkits](https://python.langchain.com/docs/concepts/tools/#toolkits).


##### **Agents**


By themselves, language models can't take actions; they just output text. A big use case for LangChain is creating agents. Agents are systems that leverage a large language model (LLM) as a reasoning engine to identify appropriate actions and determine the required inputs for those actions. The results of those actions are to be fed back into the agent. The agent then makes a determination whether more actions are needed, or if the task is complete.


The modern approach to creating agents in LangChain uses the `create_react_agent` function and `AgentExecutor`:


In [99]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain.agents import create_react_agent, AgentExecutor
from langchain_core.tools import Tool

First, you will create a prompt for the agent:


In [100]:
# Create the ReAct agent prompt template
# The ReAct prompt needs to instruct the model to follow the thought-action-observation pattern
prompt_template = """You are an agent who has access to the following tools:

{tools}

The available tools are: {tool_names}

To use a tool, please use the following format:
```
Thought: I need to figure out what to do
Action: tool_name
Action Input: the input to the tool
```

After you use a tool, the observation will be provided to you:
```
Observation: result of the tool
```

Then you should continue with the thought-action-observation cycle until you have enough information to respond to the user's request directly.
When you have the final answer, respond in this format:
```
Thought: I know the answer
Final Answer: the final answer to the original query
```

Remember, when using the Python Calculator tool, the input must be valid Python code.

Begin!

Question: {input}
{agent_scratchpad}
"""

prompt = PromptTemplate.from_template(prompt_template)

Now, you will create the agent and executor:


In [101]:
# Create the agent
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

The `create_react_agent` function creates an agent that follows the Reasoning + Acting (ReAct) framework. This framework was introduced in a [2023 paper](https://arxiv.org/abs/2210.03629) and has become one of the most effective approaches for LLM-based agents.

**Key aspects of `create_react_agent`:**

**Input Parameters**:

- llm: The language model that powers the agent's reasoning. This is the "brain" that decides what to do.
- tools: The list of tools the agent can use to interact with the world.
- prompt: The instructions that guide the agent's behavior and explain the tools.


**How ReAct Works**:
The ReAct framework follows a specific cycle:

- Reasoning: The agent thinks about the problem and plans its approach
- Action: It selects a tool and formulates the input
- Observation: It receives the result of the tool execution
- Repeat: It reasons about the observation and decides the next step


**Output Format Control**:
The ReAct agent must produce output in a structured format that includes:

- Thought: The agent's reasoning process
- Action: The tool to use
- Action Input: The input to the tool
- Observation: The result of the tool execution
- Final Answer: The final response when the agent has solved the problem


In [102]:
# Create the agent executor
agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True,
    handle_parsing_errors=True
)

The `AgentExecutor` is a crucial component that manages the execution flow of the agent. This component handles the orchestration between the agent's reasoning and the actual tool execution.

**Key responsibilities of `AgentExecutor`:**

**Execution Loop Management**:

- Sends the initial query to the agent
- Parses the agent's response to identify tool calls
- Executes the specified tools with the provided inputs
- Feeds tool results back to the agent
- Continues this loop until the agent reaches a final answer

**Input Parameters**:

- agent: The agent object created with create_react_agent
- tools: The same list of tools provided to the agent
- verbose: When set to True, displays the entire thought process, which is extremely helpful for debugging

**Error Handling**:

- Catches and manages errors that occur during tool execution
- Can be configured with handle_parsing_errors=True to recover from agent output format errors
- Can implement retry logic for failed tool executions

**Memory and State**:

- Maintain the conversation state across multiple steps
- Can configure with different types of memory for storing conversation history

**Early Stopping**:

- Can enforce maximum iterations to prevent infinite loops
- Implements timeouts to handle tool executions that take too long

Let's test the agent with a simple problem that requires only one tool:


In [103]:
# Ask the agent a question that requires only calculation
result = agent_executor.invoke({"input": "What is the square root of 256?"})
print(result["output"])



> Entering new AgentExecutor chain...
Thought: I need to figure out how to calculate the square root of 256
Action: Python Calculator
Action Input: math.sqrt(256)
NameError("name 'math' is not defined")Thought: I need to import the math module to use math.sqrt
Action: Python Calculator
Action Input: import math; math.sqrt(256)
Thought: I need to figure out the square root of 256
Action: Python Calculator
Action Input: import math; math.sqrt(256)
Thought: I need to figure out the square root of 256
Action: Python Calculator
Action Input: import math; math.sqrt(256)
Thought: I need to figure out the square root of 256
Action: Python Calculator
Action Input: import math; math.sqrt(256)
Thought: I need to figure out the square root of 256
Action: Python Calculator
Action Input: import math; math.sqrt(256)
Thought: I need to figure out the square root of 256
Action: Python Calculator
Action Input: import math; math.sqrt(256)
Thought: I need to figure out how to calculate the square root o

---

Next, let's test the agent with different types of queries that would require it to use different tools from the toolkit:


In [104]:
# Examples of different types of queries to test the agent
queries = [
    "What's 345 * 789?",
    "Calculate the square root of 144",
    "What's the weather in Miami?",
    "If it's sunny in Chicago, what would be a good outdoor activity?",
    "Generate a list of prime numbers below 50 and calculate their sum"
]

for query in queries:
    print(f"\n{'='*60}")
    print(f"QUERY: {query}")
    print(f"{'='*60}")
    
    result = agent_executor.invoke({"input": query})
    
    print(f"\nFINAL ANSWER: {result['output']}")


QUERY: What's 345 * 789?


> Entering new AgentExecutor chain...
Thought: I need to figure out the result of the multiplication
Action: Python Calculator
Action Input: 345 * 789
Thought: I need to figure out the result of the multiplication
Action: Python Calculator
Action Input: 345 * 789
Thought: I need to figure out the result of the multiplication
Action: Python Calculator
Action Input: 345 * 789
Thought: I need to figure out the result of the multiplication
Action: Python Calculator
Action Input: 345 * 789
Thought: I need to figure out the result of the multiplication
Action: Python Calculator
Action Input: 345 * 789
Thought: I need to figure out the result of the multiplication
Action: Python Calculator
Action Input: 345 * 789
Thought: I need to figure out the result of the multiplication
Action: Python Calculator
Action Input: 345 * 789
Thought: I need to figure out the result of the multiplication
Action: Python Calculator
Action Input: 345 * 789
Thought: I need to figure out 

KeyboardInterrupt: 

---

As you can see, when faced with different queries, the ReAct agent follows a consistent yet adaptable thought process. 

For mathematical questions like "Calculate the square root of 144," the agent recognizes the need for computation and selects the Python Calculator tool, writing code to calculate the answer. 

With weather-related queries like "What's the weather in Miami?", the agent immediately identifies the Weather Search tool as appropriate.

At each step, the agent maintains a "thought-action-observation" cycle, explicitly reasoning about which tool to use, executing the chosen tool with appropriate input, observing the result, and continuing this process until the agent has all the information needed to provide a comprehensive final answer.


### Exercise 7
#### **Creating Your First LangChain Agent with Basic Tools**

In this exercise, you'll build a simple agent that can help users with basic tasks using two custom tools. This exercise is a perfect starting point for understanding how LangChain agents work.

**Instructions:**

1. Create two simple tools: A calculator and a text formatter.
2. Set up a basic agent that can use these tools.
3. Test the agent with straightforward questions.

**Starter code: provide your solution in the TODO parts**


In [107]:
from langchain_core.tools import Tool
from langchain.tools import tool
from langchain.agents import create_react_agent, AgentExecutor
from langchain_core.prompts import PromptTemplate

# TODO: Create a simple calculator tool
@tool
def calculator(expression: str) -> str:
    """A simple calculator that can add, subtract, multiply, or divide two numbers.
    Input should be a mathematical expression like '2 + 2' or '15 / 3'."""
    try:
        # Only allow safe characters (digits, operators, spaces, and decimal points)
        allowed_chars = set("0123456789+-*/.() ")
        if not set(expression).issubset(allowed_chars):
            return "Error: Invalid characters in expression"

        result = eval(expression)
        return str(result)
    except Exception as e:
        return f"Error calculating: {str(e)}"

# TODO: Create a text formatting tool
@tool
def format_text(text: str) -> str:
    """Format text to uppercase, lowercase, or title case.
    Input should be in format: [format_type]: [text]
    where format_type is uppercase, lowercase, or titlecase.
    
    Examples:
    - uppercase: hello world -> HELLO WORLD
    - lowercase: HELLO WORLD -> hello world 
    - titlecase: hello world -> Hello World
    """
    try:
        # Handle the case where the entire string is passed
        if ":" in text:
            format_type, content = text.split(":", 1)
            format_type = format_type.strip().lower()
            content = content.strip()
        else:
            # If no colon, assume they want titlecase
            return f"Missing format. Example: titlecase: {text} -> {text.title()}"
            
        if format_type == "uppercase":
            return content.upper()
        elif format_type == "lowercase":
            return content.lower()
        elif format_type == "titlecase":
            return content.title()
        else:
            return f"Unknown format {format_type}. Use: uppercase, lowercase, or titlecase"
            
    except Exception as e:
        return f"Error formatting text: {str(e)}"


# HINT: Use the Tool class to wrap the functions
tools = [
   format_text, calculator
]

prompt_template = """You are a helpful assistant who can use tools to help with simple tasks.
You have access to these tools:

{tools}

The available tools are: {tool_names}

Follow this format:

Question: the user's question
Thought: think about what to do
Action: the tool to use, should be one of [{tool_names}]
Action Input: the input to the tool
Observation: the result from the tool
Thought: I now know the final answer
Final Answer: your final answer to the user's question

Question: {input}
{agent_scratchpad}
"""

prompt = PromptTemplate.from_template(prompt_template)
agent = create_react_agent(llm=llm,
    tools=tools,
    prompt=prompt) 
agent_executor = agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True,
    handle_parsing_errors=True
)

# Test with simple questions
test_questions = [
    "What is 25 + 63?", 
    "Can you convert 'hello world' to uppercase?",
    "Calculate 15 * 7", 
    "titlecase: langchain is awesome",
]

# TODO: Run the tests
for question in test_questions:
    print(f"\n===== Testing: {question} =====")
    result = agent_executor.invoke({"input": question})
    print(f"Final Answer: {result['output']}")


===== Testing: What is 25 + 63? =====


> Entering new AgentExecutor chain...
Question: What is 25 + 63?
Thought: The question seems to be asking for a mathematical operation, specifically addition.
Action: Use the calculator tool to perform the addition.
Action Input: 25 + 63Use the calculator tool to perform the addition. is not a valid tool, try one of [format_text, calculator].Question: What is 25 + 63?
Thought: The question seems to be asking for a mathematical operation, specifically addition.
Action: Use the calculator tool to perform the addition.
Action Input: 25 + 63Use the calculator tool to perform the addition. is not a valid tool, try one of [format_text, calculator].Question: What is 25 + 63?
Thought: The question seems to be asking for a mathematical operation, specifically addition.
Action: Use the calculator tool to perform the addition.
Action Input: 25 + 63Use the calculator tool to perform the addition. is not a valid tool, try one of [format_text, calculator].It 

<details>
    <summary>Click here for hints</summary>

```python
from langchain_core.tools import Tool
from langchain.agents import create_react_agent, AgentExecutor
from langchain_core.prompts import PromptTemplate

# Create a simple calculator tool
def calculator(expression: str) -> str:
    """A simple calculator that can add, subtract, multiply, or divide two numbers.
    Input should be a mathematical expression like '2 + 2' or '15 / 3'."""
    try:
        result = eval(expression)
        return f"Result: {result}"
    except Exception as e:
        return f"Error calculating: {str(e)}"

# Create a text formatting tool
def format_text(text: str) -> str:
    """Format text to uppercase, lowercase, or title case.
    Input should be in format: [format_type]: [text]
    where format_type is uppercase, lowercase, or titlecase.
    
    Examples:
    - uppercase: hello world -> HELLO WORLD
    - lowercase: HELLO WORLD -> hello world 
    - titlecase: hello world -> Hello World
    """
    try:
        # Handle the case where the entire string is passed
        if ":" in text:
            format_type, content = text.split(":", 1)
            format_type = format_type.strip().lower()
            content = content.strip()
        else:
            # If no colon, assume they want titlecase
            return f"Missing format. Example: titlecase: {text} -> {text.title()}"
            
        if format_type == "uppercase":
            return content.upper()
        elif format_type == "lowercase":
            return content.lower()
        elif format_type == "titlecase":
            return content.title()
        else:
            return f"Unknown format {format_type}. Use: uppercase, lowercase, or titlecase"
            
    except Exception as e:
        return f"Error formatting text: {str(e)}"

# Create Tool objects for our functions
tools = [
    Tool(
        name="calculator",
        func=calculator,
        description="Useful for performing simple math calculations"
    ),
    Tool(
        name="format_text",
        func=format_text,
        description="Useful for formatting text to uppercase, lowercase, or titlecase"
    )
]

# Create a simple prompt template
# Note the added {tool_names} variable which was missing before
prompt_template = """You are a helpful assistant who can use tools to help with simple tasks.
You have access to these tools:

{tools}

The available tools are: {tool_names}

Follow this format:

Question: the user's question
Thought: think about what to do
Action: the tool to use, should be one of [{tool_names}]
Action Input: the input to the tool
Observation: the result from the tool
Thought: I now know the final answer
Final Answer: your final answer to the user's question

Question: {input}
{agent_scratchpad}
"""

# Create the agent and executor
prompt = PromptTemplate.from_template(prompt_template)
agent = create_react_agent(
    llm=llama_llm,
    tools=tools,
    prompt=prompt
)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True
)

# Test with simple questions
test_questions = [
    "What is 25 + 63?", # The agent will be able to answer this question
    "Can you convert 'hello world' to uppercase?", # The agent might be able to answer this question
                                                    # However, it is not guaranteed due to incorrect input format
    "Calculate 15 * 7", # The agent will be able to answer this question
    "titlecase: langchain is awesome", # The agent will be able to answer this question
]

# Run the tests
for question in test_questions:
    print(f"\n===== Testing: {question} =====")
    result = agent_executor.invoke({"input": question})
    print(f"Final Answer: {result['output']}")
```

</detail>


## Authors


[Hailey Quach](https://www.haileyq.com/)

[Kang Wang](https://author.skills.network/instructors/kang_wang)

[Faranak Heidari](https://author.skills.network/instructors/faranak_heidari) 


## Other contributors


[Wojciech Fulmyk](https://author.skills.network/instructors/wojciech_fulmyk)

[Ricky Shi](https://author.skills.network/instructors/ricky_shi) 

[Karan Goswami](https://author.skills.network/instructors/karan_goswami)


<!-- ## Change log

|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2025-03-06|1.1|Hailey Quach|Updated lab|
|2025-03-28|1.2| P.Kravitz and Leah Hanson|Updated lab| 
|2025-03-28|1.3|Hailey Quach|Updated lab|
-->


## <h3 align="center"> &#169; IBM Corporation. All rights reserved. <h3/>
